In [ ]:
#Code(1)
#!pip install name_matching
#!pip install jieba
#!pip install cleanco
import pandas as pd
import json
import re
import string
from name_matching.name_matcher import NameMatcher
from cleanco import basename


# 读取数据
with open('bd_companies_international.jsonl', 'r', encoding='utf-8') as file:
    data = [json.loads(line) for line in file]

df_companies_a = pd.DataFrame([{'Company name': item['companyName']} for item in data])

df_companies_b = pd.DataFrame([{'name': item['requiredSearchStrings'][0] if 'requiredSearchStrings' in item and item['requiredSearchStrings'] else ''} for item in data])

df_aliases = pd.DataFrame([{'aliases': item['aliases'][0] if 'aliases' in item and item['aliases'] else ''} for item in data])

matcher = NameMatcher(number_of_matches=1, 
                      legal_suffixes=True, 
                      common_words=False, 
                      top_n=50, 
                      verbose=True)

matcher.set_distance_metrics(['ssk', 'discounted_levenshtein', 'fuzzy_wuzzy_partial_string'])

matcher.load_and_process_master_data(column='Company name',
                                     df_matching_data=df_companies_a, 
                                     transform=True)

def preprocess(text):
    # 删除所有大写字母并将其转换为小写
    text = text.lower()
    # 替换非 ASCII 字符
    text = text.encode('ascii', 'ignore').decode('ascii')
    # 删除标点符号
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    # 使用 cleanco 删除合法商业后缀
    text = basename(text)
    # 删除最常见的单词（例如"inc", "ltd", 等等）
    common_words = ['inc', 'ltd', 'corp', 'llc', 'co', 'company']
    for word in common_words:
        text = re.sub(r'\b{}\b'.format(re.escape(word)), '', text)
    # 删除多余的空格
    text = ' '.join(text.split())
    return text

user_input = input("请输入公司名进行搜索: ")
processed_input = preprocess(user_input)

df_temp = pd.DataFrame([{'name': processed_input}])

matches = matcher.match_names(to_be_matched=df_temp, 
                              column_matching='name')

if not matches.empty:
    match_index = matches.iloc[0]['match_index']
    company_name = df_companies_a.iloc[match_index]['Company name']
    print(f"匹配的公司名称: {company_name}")
else:
    print("未找到匹配的公司名称")


请输入公司名进行搜索:  apple


preprocessing...

preprocessing complete 
 searching for matches...



100%|██████████| 1/1 [00:00<00:00, 421.45it/s]


possible matches found   
 fuzzy matching...



100%|██████████| 1/1 [00:00<00:00, 106.04it/s]

done
匹配的公司名称: lapple


In [ ]:
#Code(2)
import pandas as pd
import json
import re
import string
from name_matching.name_matcher import NameMatcher
from cleanco import basename

# 首先，调用拆分代码生成CSV文件
def parse_jsonl_to_csv(input_path, output_path):
    with open(input_path, 'r', encoding='utf-8') as jsonl_file:
        with open(output_path, 'w', newline='', encoding='utf-8') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(['Company Name', 'Search String'])  # 写入表头

            for line in jsonl_file:
                data = json.loads(line)  # 将每行的JSON字符串转换为字典
                company_name = data.get('companyName', '')
                required_search_strings = data.get('requiredSearchStrings', [])
                aliases = data.get('aliases', [])

                # 将requiredSearchStrings和aliases合并，并去除重复项
                all_strings = list(set(required_search_strings + aliases))

                # 为每个字符串写入一行
                for string in all_strings:
                    csv_writer.writerow([company_name, string])

# 调用拆分函数
input_file_path = 'bd_companies_international.jsonl'
output_csv_path = 'output_companies.csv'
parse_jsonl_to_csv(input_file_path, output_csv_path)

# 读取生成的CSV文件
df_companies = pd.read_csv(output_csv_path)

# 接下来，使用名称匹配代码进行匹配
matcher = NameMatcher(number_of_matches=1, 
                      legal_suffixes=True, 
                      common_words=False, 
                      top_n=50, 
                      verbose=True)

matcher.set_distance_metrics(['ssk', 'discounted_levenshtein', 'fuzzy_wuzzy_partial_string'])

# 这里我们假设df_companies中的'Company Name'列是我们要匹配的主数据
matcher.load_and_process_master_data(column='Company Name',
                                     df_matching_data=df_companies, 
                                     transform=True)

def preprocess(text):
    text = text.lower()
    text = text.encode('ascii', 'ignore').decode('ascii')
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    text = basename(text)
    common_words = ['inc', 'ltd', 'corp', 'llc', 'co', 'company']
    for word in common_words:
        text = re.sub(r'\b{}\b'.format(re.escape(word)), '', text)
    text = ' '.join(text.split())
    return text

# 用户输入
user_input = input("请输入公司名进行搜索: ")
processed_input = preprocess(user_input)

df_temp = pd.DataFrame([{'Search String': processed_input}])

matches = matcher.match_names(to_be_matched=df_temp, 
                              column_matching='Search String')

if not matches.empty:
    match_index = matches.iloc[0]['match_index']
    company_name = df_companies.iloc[match_index]['Company Name']
    print(f"匹配的公司名称: {company_name}")
else:
    print("未找到匹配的公司名称")

请输入公司名进行搜索:  Apple


preprocessing...

preprocessing complete 
 searching for matches...



100%|██████████| 1/1 [00:00<00:00, 207.04it/s]


possible matches found   
 fuzzy matching...



100%|██████████| 1/1 [00:00<00:00, 99.40it/s]

done
匹配的公司名称: lapple


In [ ]:
#调试将requiredSearchStrings和aliases拆分
import json
import csv

# 假设你的JSONL文件路径是'bd_companies_international.jsonl'
input_file_path = 'bd_companies_international.jsonl'
output_csv_path = 'output_companies.csv'

# 读取JSONL文件并解析数据
def parse_jsonl_to_csv(input_path, output_path):
    with open(input_path, 'r', encoding='utf-8') as jsonl_file:
        with open(output_path, 'w', newline='', encoding='utf-8') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(['Company Name', 'Search String'])  # 写入表头

            for line in jsonl_file:
                data = json.loads(line)  # 将每行的JSON字符串转换为字典
                company_name = data.get('companyName', '')
                required_search_strings = data.get('requiredSearchStrings', [])
                aliases = data.get('aliases', [])

                # 将requiredSearchStrings和aliases合并，并去除重复项
                all_strings = list(set(required_search_strings + aliases))

                # 为每个字符串写入一行
                for string in all_strings:
                    csv_writer.writerow([company_name, string])

# 调用函数
parse_jsonl_to_csv(input_file_path, output_csv_path)

print(f'CSV file has been created at {output_csv_path}')

CSV file has been created at output_companies.csv


In [ ]:
#通过翻译将中文转换英文再匹配（此方法不太可行）
import pandas as pd
import json
import re
from name_matching.name_matcher import NameMatcher
from translate import Translator

# 读取jsonl文件
with open('bd_companies_international.jsonl', 'r', encoding='utf-8') as file:
    data = [json.loads(line) for line in file]

# 定义一个函数，用于判断输入的是中文还是英文
def is_chinese(input_string):
    for ch in input_string:
        if '\u4e00' <= ch <= '\u9fff':
            return True
    return False

# 定义一个函数，用于清理公司名称
def clean_name(name):
    # 去除标点符号和多余的空格
    name = re.sub(r'[^\w\s]', '', name)
    name = re.sub(r'\s+', ' ', name)
    return name.strip().lower()

# 创建原始公司名称数据集
df_companies_a = pd.DataFrame([{'Company name': item['companyName']} for item in data])

# 创建经过修改的名称数据集
df_companies_b = pd.DataFrame([{'name': item['requiredSearchStrings'][0] if 'requiredSearchStrings' in item and item['requiredSearchStrings'] else ''} for item in data])

# 创建别名数据集
df_aliases = pd.DataFrame([{'aliases': item['aliases'][0] if 'aliases' in item and item['aliases'] else ''} for item in data])

# 初始化名称匹配器
matcher = NameMatcher(number_of_matches=1, 
                      legal_suffixes=True, 
                      common_words=False, 
                      top_n=50, 
                      verbose=True)

# 调整使用的匹配度量
matcher.set_distance_metrics(['bag', 'typo', 'refined_soundex'])

# 加载需要匹配的数据
matcher.load_and_process_master_data(column='Company name',
                                     df_matching_data=df_companies_a, 
                                     transform=True)

# 初始化翻译器
translator = Translator(to_lang="en", from_lang="zh")

# 接受用户输入的公司名称
user_input = input("请输入公司名进行搜索: ")
cleaned_user_input = clean_name(user_input)

if is_chinese(user_input):
    # 翻译中文输入为英文
    translated_input = translator.translate(user_input)
    cleaned_translated_input = clean_name(translated_input)
    print(translated_input)
    df_temp = pd.DataFrame([{'name': cleaned_translated_input}])
    # 使用 NameMatcher 进行匹配
    matches = matcher.match_names(to_be_matched=df_temp, column_matching='name')
    if not matches.empty:
        match_index = matches.iloc[0]['match_index']
        company_name = df_companies_a.iloc[match_index]['Company name']
        print(f"匹配的公司名称: {company_name}")
    else:
        print("未找到匹配的公司名称")
else:
    # 使用 NameMatcher 匹配英文
    df_temp = pd.DataFrame([{'name': cleaned_user_input}])
    matches = matcher.match_names(to_be_matched=df_temp, column_matching='name')
    if not matches.empty:
        match_index = matches.iloc[0]['match_index']
        company_name = df_companies_a.iloc[match_index]['Company name']
        print(f"匹配的公司名称: {company_name}")
    else:
        print("未找到匹配的公司名称")


请输入公司名进行搜索:  apple


preprocessing...

preprocessing complete 
 searching for matches...



100%|██████████| 1/1 [00:00<00:00, 614.19it/s]


possible matches found   
 fuzzy matching...



100%|██████████| 1/1 [00:00<00:00, 120.56it/s]

done
匹配的公司名称: lapple


In [132]:
#测试Matching概率(1)（仅供测试使用）
import pandas as pd
import json
from name_matching.name_matcher import NameMatcher

# Load the data
with open('bd_companies_international.jsonl', 'r', encoding='utf-8') as file:
    data = [json.loads(line) for line in file]

# Create dataframes
df_companies_a = pd.DataFrame([{'Company name': item['companyName']} for item in data])
df_companies_b = pd.DataFrame([{'name': item['requiredSearchStrings'][0] if 'requiredSearchStrings' in item and item['requiredSearchStrings'] else ''} for item in data])
df_aliases = pd.DataFrame([{'aliases': item['aliases'][0] if 'aliases' in item and item['aliases'] else ''} for item in data])

# Initialize NameMatcher with more matches
matcher = NameMatcher(number_of_matches=7,  # Set this to the desired number of matches
                      legal_suffixes=True, 
                      common_words=False, 
                      top_n=50, 
                      verbose=True)

matcher.set_distance_metrics(['bag', 'typo', 'refined_soundex'])

# Load and process master data
matcher.load_and_process_master_data(column='Company name',
                                     df_matching_data=df_companies_a, 
                                     transform=True)

# Get user input
user_input = input("请输入公司名进行搜索: ")
df_temp = pd.DataFrame([{'name': user_input}])

# Match names
matches = matcher.match_names(to_be_matched=df_temp, 
                              column_matching='name')

# Print the DataFrame structure for debugging
print("Matches DataFrame structure:")
print(matches)

# Output matches and their scores
if not matches.empty:
    for index, row in matches.iterrows():
        if 'match_index' in row and 'matching_score' in row:
            match_index = row['match_index']
            company_name = df_companies_a.iloc[match_index]['Company name']
            score = row['matching_score']
            print(f"匹配的公司名称: {company_name}, 匹配得分: {score}")
        else:
            print(f"Row missing expected columns: {row}")
else:
    print("未找到匹配的公司名称")


请输入公司名进行搜索:  All turki


preprocessing...

preprocessing complete 
 searching for matches...



100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 235.30it/s]


possible matches found   
 fuzzy matching...



100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 50.46it/s]

done
Matches DataFrame structure:
  original_name match_name_0    score_0  match_index_0 match_name_1  \
0     all turki        altur  63.703704          48648     all tile   

     score_1  match_index_1 match_name_2   score_2  match_index_2  ...  \
0  60.740741          45780   lug turkey  56.27322          68504  ...   

  match_index_3  match_name_4    score_4 match_index_4  match_name_5  \
0          5293    bell trans  51.435329         44536      all care   

    score_5 match_index_5  match_name_6    score_6 match_index_6  
0  49.62963         45884  brisa turkey  47.222222         52189  

[1 rows x 22 columns]
Row missing expected columns: original_name       all turki
match_name_0            altur
score_0             63.703704
match_index_0           48648
match_name_1         all tile
score_1             60.740741
match_index_1           45780
match_name_2       lug turkey
score_2              56.27322
match_index_2           68504
match_name_3         digiturk
score_3     

In [ ]:
#测试Matching概率(2)（仅供测试使用）(CSV版）
import pandas as pd
import json
import re
import string
from name_matching.name_matcher import NameMatcher
from cleanco import basename

# 首先，调用拆分代码生成CSV文件
def parse_jsonl_to_csv(input_path, output_path):
    with open(input_path, 'r', encoding='utf-8') as jsonl_file:
        with open(output_path, 'w', newline='', encoding='utf-8') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(['Company Name', 'Search String'])  # 写入表头

            for line in jsonl_file:
                data = json.loads(line)  # 将每行的JSON字符串转换为字典
                company_name = data.get('companyName', '')
                required_search_strings = data.get('requiredSearchStrings', [])
                aliases = data.get('aliases', [])

                # 将requiredSearchStrings和aliases合并，并去除重复项
                all_strings = list(set(required_search_strings + aliases))

                # 为每个字符串写入一行
                for string in all_strings:
                    csv_writer.writerow([company_name, string])

# 调用拆分函数
input_file_path = 'bd_companies_international.jsonl'
output_csv_path = 'output_companies.csv'
parse_jsonl_to_csv(input_file_path, output_csv_path)

# 读取生成的CSV文件
df_companies = pd.read_csv(output_csv_path)

# 接下来，使用名称匹配代码进行匹配
matcher = NameMatcher(number_of_matches=7, 
                      legal_suffixes=True, 
                      common_words=False, 
                      top_n=50, 
                      verbose=True)

matcher.set_distance_metrics(['ssk', 'discounted_levenshtein', 'fuzzy_wuzzy_partial_string'])

# 这里我们假设df_companies中的'Company Name'列是我们要匹配的主数据
matcher.load_and_process_master_data(column='Company Name',
                                     df_matching_data=df_companies, 
                                     transform=True)

def preprocess(text):
    text = text.lower()
    text = text.encode('ascii', 'ignore').decode('ascii')
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    text = basename(text)
    common_words = ['inc', 'ltd', 'corp', 'llc', 'co', 'company']
    for word in common_words:
        text = re.sub(r'\b{}\b'.format(re.escape(word)), '', text)
    text = ' '.join(text.split())
    return text

# 用户输入
user_input = input("请输入公司名进行搜索: ")
processed_input = preprocess(user_input)

df_temp = pd.DataFrame([{'Search String': processed_input}])

matches = matcher.match_names(to_be_matched=df_temp, 
                              column_matching='Search String')

# Print the DataFrame structure for debugging
print("Matches DataFrame structure:")
print(matches)

# Output matches and their scores
if not matches.empty:
    for index, row in matches.iterrows():
        if 'match_index' in row and 'matching_score' in row:
            match_index = row['match_index']
            company_name = df_companies_a.iloc[match_index]['Company name']
            score = row['matching_score']
            print(f"匹配的公司名称: {company_name}, 匹配得分: {score}")
        else:
            print(f"Row missing expected columns: {row}")
else:
    print("未找到匹配的公司名称")

请输入公司名进行搜索:  apple


preprocessing...

preprocessing complete 
 searching for matches...



100%|██████████| 1/1 [00:00<00:00, 299.40it/s]


possible matches found   
 fuzzy matching...



100%|██████████| 1/1 [00:00<00:00, 110.14it/s]

done
Matches DataFrame structure:
  original_name match_name_0   score_0  match_index_0 match_name_1   score_1  \
0         apple       lapple  88.76547          72026       lapple  88.76547   

   match_index_1 match_name_2    score_2  match_index_2  ... match_index_3  \
0          72027      appleby  84.822854          31093  ...         49665   

   match_name_4    score_4 match_index_4  match_name_5    score_5  \
0       snapple  80.751086         97476     applebees  78.249021   

  match_index_5  match_name_6  score_6 match_index_6  
0         49666     apple inc    100.0         49648  

[1 rows x 22 columns]
Row missing expected columns: original_name        apple
match_name_0        lapple
score_0           88.76547
match_index_0        72026
match_name_1        lapple
score_1           88.76547
match_index_1        72027
match_name_2       appleby
score_2          84.822854
match_index_2        31093
match_name_3      appleone
score_3          82.824844
match_index_3        4

In [ ]:
#测试Matching概率(2)（仅供测试使用）(正式版）
import pandas as pd
import json
import re
import string
from name_matching.name_matcher import NameMatcher
from cleanco import basename

# 直接处理 JSONL 文件并转换为 DataFrame,拆分alias和requiredsearchstrings
def parse_jsonl_to_dataframe(input_path):
    data_list = []
    with open(input_path, 'r', encoding='utf-8') as jsonl_file:
        for line in jsonl_file:
            data = json.loads(line)
            company_name = data.get('companyName', '')
            required_search_strings = data.get('requiredSearchStrings', [])
            aliases = data.get('aliases', [])

            all_strings = list(set(required_search_strings + aliases))

            for string in all_strings:
                data_list.append({'Company Name': company_name, 'Search String': string})

    df = pd.DataFrame(data_list)
    return df

# 解析 JSONL 文件到 DataFrame
input_file_path = 'bd_companies_international.jsonl'
df_companies = parse_jsonl_to_dataframe(input_file_path)

# 初始化名称匹配器
matcher = NameMatcher(number_of_matches=7, 
                      legal_suffixes=True, 
                      common_words=False, 
                      top_n=50, 
                      verbose=True)

matcher.set_distance_metrics(['ssk', 'discounted_levenshtein', 'fuzzy_wuzzy_partial_string'])

# 加载并处理主数据
matcher.load_and_process_master_data(column='Company Name',
                                     df_matching_data=df_companies, 
                                     transform=True)

def preprocess(text):
    text = text.lower()
    text = text.encode('ascii', 'ignore').decode('ascii')
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    text = basename(text)
    common_words = ['inc', 'ltd', 'corp', 'llc', 'co', 'company']
    for word in common_words:
        text = re.sub(r'\b{}\b'.format(re.escape(word)), '', text)
    text = ' '.join(text.split())
    return text

# 用户输入
user_input = input("Please enter the match name: ")
processed_input = preprocess(user_input)

df_temp = pd.DataFrame([{'Search String': processed_input}])

# 进行匹配
matches = matcher.match_names(to_be_matched=df_temp, 
                              column_matching='Search String')

# 调试：输出匹配结果的结构
print("Matches DataFrame structure:")
print(matches)

# 输出匹配结果和得分
if not matches.empty:
    for index, row in matches.iterrows():
        if 'match_index' in row and 'matching_score' in row:
            match_index = row['match_index']
            company_name = df_companies.iloc[match_index]['Company Name']
            score = row['matching_score']
            print(f"匹配的公司名称: {company_name}, 匹配得分: {score}")
        else:
            print(f"Row missing expected columns: {row}")
else:
    print("未找到匹配的公司名称")


Please enter the match name:  apple


preprocessing...

preprocessing complete 
 searching for matches...



100%|██████████| 1/1 [00:00<00:00, 189.11it/s]


possible matches found   
 fuzzy matching...



100%|██████████| 1/1 [00:00<00:00, 74.20it/s]

done
Matches DataFrame structure:
  original_name match_name_0   score_0  match_index_0 match_name_1   score_1  \
0         apple       lapple  88.76547          72024       lapple  88.76547   

   match_index_1 match_name_2    score_2  match_index_2  ... match_index_3  \
0          72025      appleby  84.822854          31091  ...         49663   

   match_name_4    score_4 match_index_4  match_name_5    score_5  \
0       snapple  80.751086         97473     applebees  78.249021   

  match_index_5  match_name_6  score_6 match_index_6  
0         49664     apple inc    100.0         49646  

[1 rows x 22 columns]
Row missing expected columns: original_name        apple
match_name_0        lapple
score_0           88.76547
match_index_0        72024
match_name_1        lapple
score_1           88.76547
match_index_1        72025
match_name_2       appleby
score_2          84.822854
match_index_2        31091
match_name_3      appleone
score_3          82.824844
match_index_3        4

In [298]:
#测试Matching概率(3)(Class版)
import pandas as pd
import json
import re
import string
from name_matching.name_matcher import NameMatcher
from cleanco import basename
from fuzzychinese import FuzzyChineseMatch
from rapidfuzz import process

class CompanyNameMatcher:
    def __init__(self, english_csv='english_search_strings.csv', chinese_csv='chinese_search_strings.csv'):
        self.english_csv = english_csv
        self.chinese_csv = chinese_csv
        self.matcher = NameMatcher(number_of_matches=1, 
                                   legal_suffixes=True, 
                                   common_words=False, 
                                   top_n=50, 
                                   verbose=True)
        self.fcm = FuzzyChineseMatch(ngram_range=(3, 3), analyzer='stroke')
        
    # 用于识别CJK字符的正则表达式 
    CJK_CHARACTERS = r'\u1100-\u11ff\u2e80-\u2fff\u3040-\u31ff\u3400-\u9fff\ua960-\ua97f\uac00-\ud7ff\uf900-\ufaff'
    CJK_OR_NUMERIC_REGEX = re.compile(rf"(?P<cjk>[{CJK_CHARACTERS}]+)|(?P<numeric>((?<=^\D)|(?<=[^\W0-9_]|\s))(?<!\b[a-zA-Z])(\d+([\W_]{{0,5}}\d+){{0,5}})(?=($|[^\W0-9_]|\s)))")
    ENGLISH_SPLITTER_REGEX = re.compile(r"[^\w&_+*\\/'#\-]+")


    # 分隔英文、数字和CJK字符的函数
    @staticmethod
    def split_english_number_cjk(text, separate_return=True, split_same_language=False):
        en_number_words = []
        cjk_words = []
        start, end = 0, len(text)
        while start < end and (r_ := CJK_OR_NUMERIC_REGEX.search(text, pos=start)):
            if word := text[start:r_.start()].strip():
                if split_same_language:
                    en_number_words.extend(
                        [stripped_word for word_ in ENGLISH_SPLITTER_REGEX.split(word) if (stripped_word := word_.strip())])
                else:
                    en_number_words.append(word)
            if cjk_word := r_.groupdict().get("cjk"):
                if split_same_language:
                    (cjk_words if separate_return else en_number_words).extend(list(cjk_word))
                else:
                    (cjk_words if separate_return else en_number_words).append(cjk_word)
            else:
                numeric_word = r_.groupdict().get("numeric")
                if split_same_language:
                    en_number_words.extend([stripped_word for word_ in ENGLISH_SPLITTER_REGEX.split(numeric_word) if (stripped_word := word_.strip())])
                else:
                    en_number_words.append(numeric_word)
            start = r_.end()

        if end > start and (text := text[start:].strip()):
            if split_same_language:
                en_number_words.extend([word_ for word in ENGLISH_SPLITTER_REGEX.split(text) if (word_ := word.strip())])
            else:
                en_number_words.append(text)
        return (en_number_words, cjk_words) if separate_return else en_number_words

    # 解析JSONL文件并分隔英文和中文字符串的函数
    @staticmethod
    def parse_and_separate_jsonl(input_path):
        english_list = []
        chinese_list = []

        with open(input_path, 'r', encoding='utf-8') as jsonl_file:
            for line in jsonl_file:
                data = json.loads(line)
                company_name = data.get('companyName', '')
                required_search_strings = data.get('requiredSearchStrings', [])
                aliases = data.get('aliases', [])

                all_strings = list(set(required_search_strings + aliases))

                for string in all_strings:
                    en_strings, cjk_strings = CompanyNameMatcher.split_english_number_cjk(string, separate_return=True)
                    for en_str in en_strings:
                        english_list.append({'Company Name': company_name, 'Search String': en_str})
                    for cjk_str in cjk_strings:
                        chinese_list.append({'Company Name': company_name, 'Search String': cjk_str})

        df_english = pd.DataFrame(english_list)
        df_chinese = pd.DataFrame(chinese_list)

        return df_english, df_chinese

    # 预处理定义
    @staticmethod
    def preprocess(text):
        text = text.lower()
        text = text.encode('ascii', 'ignore').decode('ascii')
        text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
        text = basename(text)
        common_words = ['inc', 'ltd', 'corp', 'llc', 'co', 'company']
        for word in common_words:
            text = re.sub(r'\b{}\b'.format(re.escape(word)), '', text)
        text = ' '.join(text.split())
        return text

    # fuzzychinese
    def find_company_name(self, user_input):
        df = pd.read_csv(self.chinese_csv)

        search_strings = df['Search String']
        company_names = df['Company Name']

        self.fcm.fit(search_strings)
        user_input_series = pd.Series([user_input])
        top_matches = self.fcm.transform(user_input_series, n=1)

        matched_index = self.fcm.get_index()[0][0]
        matched_company_name = company_names.iloc[matched_index]
        return matched_company_name

    # 定义一个函数来匹配输入的Search String
    def match_company_name(self, input_string):
        df = pd.read_csv(self.english_csv)

        best_match = process.extractOne(input_string, df['Search String'])
        if best_match:
            matched_row = df[df['Search String'] == best_match[0]]
            return matched_row['Company Name'].values[0]
        else:
            return None

    # 加载并处理数据
    def load_and_process_data(self):
        df_english = pd.read_csv(self.english_csv)
        self.matcher.load_and_process_master_data(column='Search String', df_matching_data=df_english, transform=True)

        df_chinese = pd.read_csv(self.chinese_csv)
        search_strings = df_chinese['Search String']
        self.fcm.fit(search_strings)

    # 匹配公司名称
    def match_input(self, user_input):
        if re.search(f'[{CJK_CHARACTERS}]', user_input):
            matched_company_name = self.find_company_name(user_input)
            print(f"匹配到的公司名称是: {matched_company_name}")
        else:
            processed_input = self.preprocess(user_input)
            df_temp = pd.DataFrame([{'Search String': processed_input}])
            matches = self.matcher.match_names(to_be_matched=df_temp, column_matching='Search String')
            input_string = ('\n'.join(matches['match_name'].dropna().astype(str)))
            company_name = self.match_company_name(input_string)

            if company_name:
                print(f"匹配到的公司名称是: {company_name}")
            else:
                print("没有匹配到相应的公司名称")


def main():
    input_file_path = 'bd_companies_international.jsonl'
    matcher = CompanyNameMatcher()

    df_english, df_chinese = CompanyNameMatcher.parse_and_separate_jsonl(input_file_path)
    df_chinese.to_csv(matcher.chinese_csv, index=False, encoding='utf-8-sig')
    df_english.to_csv(matcher.english_csv, index=False, encoding='utf-8-sig')

    matcher.load_and_process_data()

    user_input = input("请输入匹配的名称: ")
    matcher.match_input(user_input)

if __name__ == "__main__":
    main()

请输入匹配的名称:  d


preprocessing...

preprocessing complete 
 searching for matches...



100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 804.59it/s]


possible matches found   
 fuzzy matching...



100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 87.40it/s]

done
匹配到的公司名称是: Union Square Hospitality Group


In [295]:
#测试Matching概率(3)
#!pip install fuzzychinese
#!pip install git+https://github.com/DanielCGL/name_matching_for_company.git
#!pip install rapidfuzz
import pandas as pd
import json
import re
import string
from name_matching.name_matcher import NameMatcher
from cleanco import basename
from fuzzychinese import FuzzyChineseMatch
from rapidfuzz import process

# 用于识别CJK字符的正则表达式 
CJK_CHARACTERS = r'\u1100-\u11ff\u2e80-\u2fff\u3040-\u31ff\u3400-\u9fff\ua960-\ua97f\uac00-\ud7ff\uf900-\ufaff'
CJK_OR_NUMERIC_REGEX = re.compile(rf"(?P<cjk>[{CJK_CHARACTERS}]+)|(?P<numeric>((?<=^\D)|(?<=[^\W0-9_]|\s))(?<!\b[a-zA-Z])(\d+([\W_]{{0,5}}\d+){{0,5}})(?=($|[^\W0-9_]|\s)))")

ENGLISH_SPLITTER_REGEX = re.compile(r"[^\w&_+*\\/'#\-]+")

# 分隔英文、数字和CJK字符的函数
def split_english_number_cjk(text, separate_return=True, split_same_language=False):
    en_number_words = []
    cjk_words = []
    start, end = 0, len(text)
    while start < end and (r_ := CJK_OR_NUMERIC_REGEX.search(text, pos=start)):
        if word := text[start:r_.start()].strip():
            if split_same_language:
                en_number_words.extend(
                    [stripped_word for word_ in ENGLISH_SPLITTER_REGEX.split(word) if (stripped_word := word_.strip())])
            else:
                en_number_words.append(word)
        if cjk_word := r_.groupdict().get("cjk"):
            if split_same_language:
                (cjk_words if separate_return else en_number_words).extend(list(cjk_word))
            else:
                (cjk_words if separate_return else en_number_words).append(cjk_word)
        else:
            numeric_word = r_.groupdict().get("numeric")
            if split_same_language:
                en_number_words.extend([stripped_word for word_ in ENGLISH_SPLITTER_REGEX.split(numeric_word) if (stripped_word := word_.strip())])
            else:
                en_number_words.append(numeric_word)
        start = r_.end()

    if end > start and (text := text[start:].strip()):
        if split_same_language:
            en_number_words.extend([word_ for word in ENGLISH_SPLITTER_REGEX.split(text) if (word_ := word.strip())])
        else:
            en_number_words.append(text)
    return (en_number_words, cjk_words) if separate_return else en_number_words

# 解析JSONL文件并分隔英文和中文字符串的函数
def parse_and_separate_jsonl(input_path):
    english_list = []
    chinese_list = []

    with open(input_path, 'r', encoding='utf-8') as jsonl_file:
        for line in jsonl_file:
            data = json.loads(line)
            company_name = data.get('companyName', '')
            required_search_strings = data.get('requiredSearchStrings', [])
            aliases = data.get('aliases', [])

            all_strings = list(set(required_search_strings + aliases))

            for string in all_strings:
                en_strings, cjk_strings = split_english_number_cjk(string, separate_return=True)
                for en_str in en_strings:
                    english_list.append({'Company Name': company_name, 'Search String': en_str})
                for cjk_str in cjk_strings:
                    chinese_list.append({'Company Name': company_name, 'Search String': cjk_str})

    df_english = pd.DataFrame(english_list)
    df_chinese = pd.DataFrame(chinese_list)

    return df_english, df_chinese

# 解析JSONL并分隔英文和中文字符串
input_file_path = 'bd_companies_international.jsonl'
df_english, df_chinese = parse_and_separate_jsonl(input_file_path)

# 保存中文字符串到CSV
df_chinese.to_csv('chinese_search_strings.csv', index=False, encoding='utf-8-sig')

# 保存英文字符串到CSV
df_english.to_csv('english_search_strings.csv', index=False, encoding='utf-8-sig')

# 初始化名称匹配器
matcher = NameMatcher(number_of_matches=1, 
                      legal_suffixes=True, 
                      common_words=False, 
                      top_n=50, 
                      verbose=True)

# 加载和处理拆分后的英文字符串数据
df_english = pd.read_csv('english_search_strings.csv')
matcher.load_and_process_master_data(column='Search String', df_matching_data=df_english, transform=True)

# 加载和处理拆分后的中文字符串数据
df_chinese = pd.read_csv('chinese_search_strings.csv')
search_strings = df_chinese['Search String']
company_names = df_chinese['Company Name']
fcm = FuzzyChineseMatch(ngram_range=(3, 3), analyzer='stroke')
fcm.fit(search_strings)

#预处理定义
def preprocess(text):
    text = text.lower()
    text = text.encode('ascii', 'ignore').decode('ascii')
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    text = basename(text)
    common_words = ['inc', 'ltd', 'corp', 'llc', 'co', 'company']
    for word in common_words:
        text = re.sub(r'\b{}\b'.format(re.escape(word)), '', text)
    text = ' '.join(text.split())
    return text

#fuzzychinese
def find_company_name(user_input, csv_file='chinese_search_strings.csv'):
    # Load your CSV file
    df = pd.read_csv(csv_file)

    # Extract the 'search string' and 'company name' columns
    search_strings = df['Search String']
    company_names = df['Company Name']

    # Initialize FuzzyChineseMatch
    fcm = FuzzyChineseMatch(ngram_range=(3, 3), analyzer='stroke')
    fcm.fit(search_strings)

    # Find the top matches for the user input
    user_input_series = pd.Series([user_input])
    top_matches = fcm.transform(user_input_series, n=1)  # Get the top 1 match

    # Retrieve the matching company name
    matched_index = fcm.get_index()[0][0]  # Index of the top match
    matched_company_name = company_names.iloc[matched_index]
    return matched_company_name

# 定义一个函数来匹配输入的Search String
def match_company_name(input_string, df):
    df = pd.read_csv('english_search_strings.csv')
    # 使用rapidfuzz的process来匹配最相近的Search String
    best_match = process.extractOne(input_string, df['Search String'])
    
    # 获取匹配到的Search String对应的Company Name
    if best_match:
        matched_row = df[df['Search String'] == best_match[0]]
        return matched_row['Company Name'].values[0]
    else:
        return None

user_input = input("请输入匹配的名称: ")

# 判断输入是否包含中文字符
if re.search(f'[{CJK_CHARACTERS}]', user_input):# 如果输入包含中文字符，使用 FuzzyChineseMatch 进行匹配
    
    # Get the matched company name
    matched_company_name = find_company_name(user_input)

    # Output the result
    print(f"匹配到的公司名称是: {matched_company_name}")
else:# 如果输入不包含中文字符，使用 NameMatcher 进行匹配
    processed_input = preprocess(user_input)
    df_temp = pd.DataFrame([{'Search String': processed_input}])
    matches = matcher.match_names(to_be_matched=df_temp, column_matching='Search String')
    # 输出 DataFrame 的列名和内容以进行调试
    print("Matches DataFrame content:", matches.head())
    #将匹配的Search String匹配与其相关联的Company Name
    input_string = ('\n'.join(matches['match_name'].dropna().astype(str)))
    company_name = match_company_name(input_string, df)

    if company_name:
        print(f"匹配到的公司名称是: {company_name}")
    else:
        print("没有匹配到相应的公司名称")

请输入匹配的名称:  oppo us


preprocessing...

preprocessing complete 
 searching for matches...



100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 250.00it/s]


possible matches found   
 fuzzy matching...



100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 70.26it/s]

done
Matches DataFrame content:   original_name match_name     score  match_index
0       oppo us   ovivo us  67.80303        99412
匹配到的公司名称是: Ovivo US


In [481]:
#测试Matching概率(4)
#!pip install fuzzychinese
#!pip install git+https://github.com/DanielCGL/name_matching_for_company.git
#!pip install rapidfuzz
import pandas as pd
import json
import re
import string
from deep_translator import GoogleTranslator
from name_matching.name_matcher import NameMatcher
from cleanco import basename
from fuzzychinese import FuzzyChineseMatch
from rapidfuzz import process

# 分隔英文、数字和CJK字符的函数
def split_english_number_cjk(text, separate_return=True, split_same_language=False):
    en_number_words = []
    cjk_words = []
    start, end = 0, len(text)
    while start < end and (r_ := CJK_OR_NUMERIC_REGEX.search(text, pos=start)):
        if word := text[start:r_.start()].strip():
            if split_same_language:
                en_number_words.extend(
                    [stripped_word for word_ in ENGLISH_SPLITTER_REGEX.split(word) if (stripped_word := word_.strip())])
            else:
                en_number_words.append(word)
        if cjk_word := r_.groupdict().get("cjk"):
            if split_same_language:
                (cjk_words if separate_return else en_number_words).extend(list(cjk_word))
            else:
                (cjk_words if separate_return else en_number_words).append(cjk_word)
        else:
            numeric_word = r_.groupdict().get("numeric")
            if split_same_language:
                en_number_words.extend([stripped_word for word_ in ENGLISH_SPLITTER_REGEX.split(numeric_word) if (stripped_word := word_.strip())])
            else:
                en_number_words.append(numeric_word)
        start = r_.end()

    if end > start and (text := text[start:].strip()):
        if split_same_language:
            en_number_words.extend([word_ for word in ENGLISH_SPLITTER_REGEX.split(text) if (word_ := word.strip())])
        else:
            en_number_words.append(text)
    return (en_number_words, cjk_words) if separate_return else en_number_words

# 清理中文字符串中不必要的空格
def clean_chinese_strings(text):
    # 匹配并删除中文字符之间的空格
    cleaned_text = re.sub(r'(?<=[\u4e00-\u9fff])\s+(?=[\u4e00-\u9fff])', '', text)
    return cleaned_text

# 解析JSONL文件并处理数据
def parse_and_clean_jsonl(input_path, output_path):
    cleaned_data = []

    with open(input_path, 'r', encoding='utf-8') as jsonl_file:
        for line in jsonl_file:
            data = json.loads(line)
            company_name = data.get('companyName', '')
            required_search_strings = data.get('requiredSearchStrings', [])
            aliases = data.get('aliases', [])

            # 清理requiredSearchStrings中的中文字符串
            cleaned_required_search_strings = [
                clean_chinese_strings(string) if CJK_REGEX.search(string) else string
                for string in required_search_strings
            ]
            
            # 清理aliases中的中文字符串
            cleaned_aliases = [
                clean_chinese_strings(alias) if CJK_REGEX.search(alias) else alias
                for alias in aliases
            ]

            # 更新数据字典
            cleaned_data.append({
                'requiredSearchStrings': cleaned_required_search_strings,
                'aliases': cleaned_aliases,
                'companyName': company_name
            })

    # 将清理后的数据写入新的JSONL文件
    with open(output_path, 'w', encoding='utf-8') as output_file:
        for item in cleaned_data:
            output_file.write(json.dumps(item, ensure_ascii=False) + '\n')

# 输入文件路径
input_file_path = 'bd_companies_international.jsonl'
output_file_path = 'cleaned_bd_companies_international.jsonl'

# 解析和清理JSONL文件
parse_and_clean_jsonl(input_file_path, output_file_path)

# 继续后续的代码...
df_english, df_chinese = parse_and_separate_jsonl(output_file_path)

# 保存中文字符串到CSV
df_chinese.to_csv('chinese_search_strings.csv', index=False, encoding='utf-8-sig')

# 保存英文字符串到CSV
df_english.to_csv('english_search_strings.csv', index=False, encoding='utf-8-sig')

# 初始化名称匹配器
matcher = NameMatcher(number_of_matches=1, 
                      legal_suffixes=True, 
                      common_words=False, 
                      top_n=50, 
                      verbose=True)

# 加载和处理拆分后的英文字符串数据
df_english = pd.read_csv('english_search_strings.csv')
matcher.load_and_process_master_data(column='Search String', df_matching_data=df_english, transform=True)

# 加载和处理拆分后的中文字符串数据
df_chinese = pd.read_csv('chinese_search_strings.csv')
search_strings = df_chinese['Search String']
company_names = df_chinese['Company Name']
fcm = FuzzyChineseMatch(ngram_range=(3, 3), analyzer='stroke')
fcm.fit(search_strings)

#预处理定义
def preprocess(text):
    """
    Preprocesses the given text by performing the following operations:
    1. Converts the text to lowercase.
    2. Removes circumflex and other diacritical marks from characters.
    3. Removes punctuation.
    4. Removes common words that may not be necessary for processing.
    5. Trims extra spaces and returns a clean, single-line string.

    Parameters:
    - text (str): The input text to preprocess.

    Returns:
    - str: The preprocessed text.
    """

    # Convert the text to lowercase
    text = text.lower()

    # Remove circumflex and other diacritical marks from characters
    text = circumflex_regulator(text)

    # Remove punctuation using a regular expression that matches all punctuation characters and replaces them with an empty string
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)

    # Apply the 'basename' function to the text to remove certain prefixes or suffixes
    text = basename(text)

    # Define a list of common words to remove from the text
    common_words = ['inc', 'ltd', 'corp', 'llc', 'co', 'company']

    # Iterate over the list of common words and remove them from the text using a regular expression that matches whole words only
    for word in common_words:
        text = re.sub(r'\b{}\b'.format(re.escape(word)), '', text)

    # Split the text into words and rejoin them with a single space to ensure there are no extra spaces in the final output
    text = ' '.join(text.split())

    # Return the preprocessed text
    return text


# CIRCUMFLEX_PAIRS Usage:
#    Remove circumflex from characters in search string
#    mapping: 大小写声调字母 -> 标准英文字母
CIRCUMFLEX_PAIRS = \
    ((r'àâáãāåäǎ', 'a'), (r'ÀÂÁÃĀÅÄǍ', 'A'), (r'ß', 'b'), (r'čćç', 'c'), (r'ČÇ', 'C'), (r'ðđ', 'd'), (r'ÐĐ', 'D'),
     (r'éěèêëėệē', 'e'), (r'ÉÈÊËĖỆĒ', 'E'), (r'ğ', 'g'), (r'Ğ', 'G'), (r'îïíīịìǐĩı', 'i'), (r'İÎÏÍĪỊÌǏ', 'I'),
     (r'Ł', 'L'), (r'ł', 'l'), (r'ñňń', 'n'), (r'ÑŇŃ', 'N'), (r'ôöōồǒóòøőõ', 'o'), (r'ÔÖŌỒÓÒØŐÕ', 'O'), (r'ŕ', 'r'),
     (r'şš', 's'), (r'ŠŞ', 'S'), (r'ť', 't'), (r'ûùüúǔưū', 'u'), (r'ÛÙÜÚƯŪ', 'U'), (r'ŵ', 'w'), (r'ý', 'y'),
     (r'Ý', 'Y'), (r'žź', 'z'), (r'ŽŹ', 'Z'), (r'æ', 'ae'), (r'Æ', 'AE'))
CIRCUMFLEX_REGEX_PAIRS = tuple(
    (re.compile(rf'[{regex_}]'), format_) for (regex_, format_) in CIRCUMFLEX_PAIRS
)

# Format circumflex to regulate English characters
def circumflex_regulator(name: str):
    if not name:
        return ''
    for regex_, format_character in CIRCUMFLEX_REGEX_PAIRS:
        name = regex_.sub(format_character, name)
    return name


#fuzzychinese
def find_company_name(user_input, csv_file='chinese_search_strings.csv'):
    # Load your CSV file
    df = pd.read_csv(csv_file)

    # Extract the 'search string' and 'company name' columns
    search_strings = df['Search String']
    company_names = df['Company Name']

    # Initialize FuzzyChineseMatch
    fcm = FuzzyChineseMatch(ngram_range=(3, 3), analyzer='char')
    fcm.fit(search_strings)

    # Find the top matches for the user input
    user_input_series = pd.Series([user_input])
    top_matches = fcm.transform(user_input_series, n=1)  # Get the top 1 match

    # Retrieve the matching company name
    matched_index = fcm.get_index()[0][0]  # Index of the top match
    matched_company_name = company_names.iloc[matched_index]
    return matched_company_name

# 定义一个函数来匹配输入的Search String
def match_company_name(input_string, df):
    df = pd.read_csv('english_search_strings.csv')
    # 使用rapidfuzz的process来匹配最相近的Search String
    best_match = process.extractOne(input_string, df['Search String'])
    
    # 获取匹配到的Search String对应的Company Name
    if best_match:
        matched_row = df[df['Search String'] == best_match[0]]
        return matched_row['Company Name'].values[0]
    else:
        return None

# 定义需要移除的中文标点符号
CHINESE_PUNCTUATION = r'[！？｡＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏.]'

user_input = input("请输入匹配的名称: ")

if any(CJK_REGEX.match(char) for char in user_input):  # 包含中文字符
    # 清理中文标点符号
    user_input = re.sub(CHINESE_PUNCTUATION, '', user_input)
    
    if all(CJK_REGEX.match(char) for char in user_input):  # 全部是中文字符
        matched_company_name = find_company_name(user_input)
        
        if matched_company_name:
            print(f"匹配到的公司名称是: {matched_company_name}")
        else:
            print("没有匹配到相应的公司名称")
    else:  # 混合中文和英文字符，或包含其他外文字符
        translated_input = GoogleTranslator(source='auto', target='en').translate(user_input)
        processed_input = preprocess(translated_input)
        df_temp = pd.DataFrame([{'Search String': processed_input}])
        matches = matcher.match_names(to_be_matched=df_temp, column_matching='Search String')
        input_string = ('\n'.join(matches['match_name'].dropna().astype(str)))
        
        company_name = match_company_name(input_string, df_english)
        
        input_score = ('\n'.join(matches['score'].dropna().astype(str)))
        input_score = float(input_score)

        if input_score >= 85:
            print(f"匹配到的公司名称是: {company_name}")
        else:
            print("没有匹配到相应的公司名称")
else:  # 全部是英文字符
    processed_input = preprocess(user_input)
    df_temp = pd.DataFrame([{'Search String': processed_input}])
    matches = matcher.match_names(to_be_matched=df_temp, column_matching='Search String')
    input_string = ('\n'.join(matches['match_name'].dropna().astype(str)))
    
    company_name = match_company_name(input_string, df_english)
    
    input_score = ('\n'.join(matches['score'].dropna().astype(str)))
    input_score = float(input_score)
    if input_score >= 85:
        print(f"匹配到的公司名称是: {company_name}")
    else:
        print("没有匹配到相应的公司名称")

preprocessing...

preprocessing complete 
 searching for matches...



100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 602.46it/s]


possible matches found   
 fuzzy matching...



100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 72.89it/s]

done
没有匹配到相应的公司名称


请输入匹配的名称:  j


In [8]:
# 测试Matching概率(5)
# !pip install fuzzychinese
# !pip install git+https://github.com/DanielCGL/name_matching_for_company.git
# !pip install rapidfuzz
import pandas as pd
import json
import re
import string
from deep_translator import GoogleTranslator
from name_matching.name_matcher import NameMatcher
from cleanco import basename
from fuzzychinese import FuzzyChineseMatch
from rapidfuzz import process

CJK_REGEX = re.compile(r'[\u4e00-\u9fff]')
CJK_OR_NUMERIC_REGEX = re.compile(r'(?P<cjk>[\u4E00-\u9FFF]+)|(?P<numeric>[0-9]+)')
ENGLISH_SPLITTER_REGEX = re.compile(r'\s+')

# 分隔英文、数字和CJK字符的函数
def split_english_number_cjk(text, separate_return=True, split_same_language=False):
    en_number_words = []
    cjk_words = []
    start, end = 0, len(text)
    while start < end and (r_ := CJK_OR_NUMERIC_REGEX.search(text, pos=start)):
        if word := text[start:r_.start()].strip():
            en_number_words.append(word)
        if cjk_word := r_.groupdict().get("cjk"):
            cjk_words.append(cjk_word)
        else:
            numeric_word = r_.groupdict().get("numeric")
            en_number_words.append(numeric_word)
        start = r_.end()

    if end > start and (text := text[start:].strip()):
        en_number_words.append(text)
    return (en_number_words, cjk_words) if separate_return else en_number_words



# 清理中文字符串中不必要的空格
def clean_chinese_strings(text):
    # 匹配并删除中文字符之间的空格
    cleaned_text = re.sub(r'(?<=[\u4e00-\u9fff])\s+(?=[\u4e00-\u9fff])', '', text)
    return cleaned_text


# 解析JSONL文件并处理数据
def parse_and_clean_jsonl(input_path, output_path):
    cleaned_data = []

    with open(input_path, 'r', encoding='utf-8') as jsonl_file:
        for line in jsonl_file:
            data = json.loads(line)
            company_name = data.get('companyName', '')
            required_search_strings = data.get('requiredSearchStrings', [])
            aliases = data.get('aliases', [])

            # 清理requiredSearchStrings中的中文字符串
            cleaned_required_search_strings = [
                clean_chinese_strings(string) if CJK_REGEX.search(string) else string
                for string in required_search_strings
            ]

            # 清理aliases中的中文字符串
            cleaned_aliases = [
                clean_chinese_strings(alias) if CJK_REGEX.search(alias) else alias
                for alias in aliases
            ]

            # 更新数据字典
            cleaned_data.append({
                'requiredSearchStrings': cleaned_required_search_strings,
                'aliases': cleaned_aliases,
                'companyName': company_name
            })

    # 将清理后的数据写入新的JSONL文件
    with open(output_path, 'w', encoding='utf-8') as output_file:
        for item in cleaned_data:
            output_file.write(json.dumps(item, ensure_ascii=False) + '\n')


# 使用修改后的函数进行解析
def parse_and_separate_jsonl(input_path):
    english_list = []
    chinese_list = []

    with open(input_path, 'r', encoding='utf-8') as jsonl_file:
        for line in jsonl_file:
            data = json.loads(line)
            company_name = data.get('companyName', '')
            required_search_strings = data.get('requiredSearchStrings', [])
            aliases = data.get('aliases', [])

            all_strings = list(set(required_search_strings + aliases))

            for string in all_strings:
                en_strings, cjk_strings = split_english_number_cjk(string, separate_return=True, split_same_language=False)
                for en_str in en_strings:
                    english_list.append({'Company Name': company_name, 'Search String': en_str})
                for cjk_str in cjk_strings:
                    chinese_list.append({'Company Name': company_name, 'Search String': cjk_str})

    df_english = pd.DataFrame(english_list)
    df_chinese = pd.DataFrame(chinese_list)

    return df_english, df_chinese


# 输入文件路径
input_file_path = 'bd_companies_international.jsonl'
output_file_path = 'cleaned_bd_companies_international.jsonl'

# 解析和清理JSONL文件
parse_and_clean_jsonl(input_file_path, output_file_path)

# 继续后续的代码...
df_english, df_chinese = parse_and_separate_jsonl(output_file_path)

# 保存中文字符串到CSV
df_chinese.to_csv('chinese_search_strings.csv', index=False, encoding='utf-8-sig')

# 保存英文字符串到CSV
df_english.to_csv('english_search_strings.csv', index=False, encoding='utf-8-sig')

# 初始化名称匹配器
matcher = NameMatcher(number_of_matches=1,
                      legal_suffixes=True,
                      common_words=False,
                      top_n=50,
                      verbose=True)

# 加载和处理拆分后的英文字符串数据
df_english = pd.read_csv('english_search_strings.csv')
matcher.load_and_process_master_data(column='Search String', df_matching_data=df_english, transform=True)

# 加载和处理拆分后的中文字符串数据
df_chinese = pd.read_csv('chinese_search_strings.csv')
search_strings = df_chinese['Search String']
company_names = df_chinese['Company Name']


# 预处理定义
def preprocess(text):
    # Convert the text to lowercase
    text = text.lower()

    # Remove circumflex and other diacritical marks from characters
    text = circumflex_regulator(text)

    # Remove punctuation using a regular expression that matches all punctuation characters and replaces them with an empty string
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)

    # Apply the 'basename' function to the text to remove certain prefixes or suffixes
    text = basename(text)

    # Define a list of common words to remove from the text
    common_words = ['inc', 'ltd', 'corp', 'llc', 'co', 'company']

    # Iterate over the list of common words and remove them from the text using a regular expression that matches whole words only
    for word in common_words:
        text = re.sub(r'\b{}\b'.format(re.escape(word)), '', text)

    # Split the text into words and rejoin them with a single space to ensure there are no extra spaces in the final output
    text = ' '.join(text.split())

    # Return the preprocessed text
    return text


# CIRCUMFLEX_PAIRS Usage:
#    Remove circumflex from characters in search string
#    mapping: 大小写声调字母 -> 标准英文字母
CIRCUMFLEX_PAIRS = \
    ((r'àâáãāåäǎ', 'a'), (r'ÀÂÁÃĀÅÄǍ', 'A'), (r'ß', 'b'), (r'čćç', 'c'), (r'ČÇ', 'C'), (r'ðđ', 'd'), (r'ÐĐ', 'D'),
     (r'éěèêëėệē', 'e'), (r'ÉÈÊËĖỆĒ', 'E'), (r'ğ', 'g'), (r'Ğ', 'G'), (r'îïíīịìǐĩı', 'i'), (r'İÎÏÍĪỊÌǏ', 'I'),
     (r'Ł', 'L'), (r'ł', 'l'), (r'ñňń', 'n'), (r'ÑŇŃ', 'N'), (r'ôöōồǒóòøőõ', 'o'), (r'ÔÖŌỒÓÒØŐÕ', 'O'), (r'ŕ', 'r'),
     (r'şš', 's'), (r'ŠŞ', 'S'), (r'ť', 't'), (r'ûùüúǔưū', 'u'), (r'ÛÙÜÚƯŪ', 'U'), (r'ŵ', 'w'), (r'ý', 'y'),
     (r'Ý', 'Y'), (r'žź', 'z'), (r'ŽŹ', 'Z'), (r'æ', 'ae'), (r'Æ', 'AE'))
CIRCUMFLEX_REGEX_PAIRS = tuple(
    (re.compile(rf'[{regex_}]'), format_) for (regex_, format_) in CIRCUMFLEX_PAIRS
)


# Format circumflex to regulate English characters
def circumflex_regulator(name: str):
    if not name:
        return ''
    for regex_, format_character in CIRCUMFLEX_REGEX_PAIRS:
        name = regex_.sub(format_character, name)
    return name


# fuzzychinese
def find_company_name(user_input, csv_file='chinese_search_strings.csv'):
    # 加载CSV文件
    df = pd.read_csv(csv_file)

    # 提取'search string' 和 'company name' 列
    search_strings = df['Search String']
    company_names = df['Company Name']

    # 尝试不同的ngram_range
    fcm = FuzzyChineseMatch(ngram_range=(2, 3), analyzer='char')
    fcm.fit(search_strings)

    # 找到用户输入的最佳匹配
    user_input_series = pd.Series([user_input])
    top_matches = fcm.transform(user_input_series, n=1)  # 获取前1个匹配

    # 获取最佳匹配的索引
    matched_index = fcm.get_index()[0][0]
    matched_company_name = company_names.iloc[matched_index]

    # 获取匹配分数
    similarity_scores = fcm.get_similarity_score()

    # 获取第一个匹配的分数
    top1_score = similarity_scores[0][0]

    return matched_company_name, top1_score


# 定义一个函数来匹配输入的Search String
def match_company_name(input_string, df):
    df = pd.read_csv('english_search_strings.csv')
    # 使用rapidfuzz的process来匹配最相近的Search String
    best_match = process.extractOne(input_string, df['Search String'])

    # 获取匹配到的Search String对应的Company Name
    if best_match:
        matched_row = df[df['Search String'] == best_match[0]]
        return matched_row['Company Name'].values[0]
    else:
        return None


# 定义需要移除的中文标点符号
CHINESE_PUNCTUATION = r'[！？｡＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏.]'
user_input = input("请输入匹配的名称: ")
if any(CJK_REGEX.match(char) for char in user_input):  # 包含中文字符
    # 清理中文标点符号
    user_input = re.sub(CHINESE_PUNCTUATION, '', user_input)

    if all(CJK_REGEX.match(char) for char in user_input):  # 全部是中文字符
        matched_company_name, top1_score = find_company_name(user_input)

        if top1_score >= 0.6:
            print(f"匹配到的公司名称是: {matched_company_name}, 匹配得分是: {top1_score}")
        else:
            print("没有匹配到相应的公司名称")
    else:  # 混合中文和英文字符，或包含其他外文字符
        translated_input = GoogleTranslator(source='auto', target='en').translate(user_input)
        processed_input = preprocess(translated_input)
        df_temp = pd.DataFrame([{'Search String': processed_input}])
        matches = matcher.match_names(to_be_matched=df_temp, column_matching='Search String')
        input_string = ('\n'.join(matches['match_name'].dropna().astype(str)))

        company_name = match_company_name(input_string, df_english)

        input_score = ('\n'.join(matches['score'].dropna().astype(str)))
        input_score = float(input_score)

        if input_score >= 85:
            print(f"匹配到的公司名称是: {company_name}")
        else:
            print("没有匹配到相应的公司名称")
else:  # 全部是英文字符
    processed_input = preprocess(user_input)
    df_temp = pd.DataFrame([{'Search String': processed_input}])
    matches = matcher.match_names(to_be_matched=df_temp, column_matching='Search String')
    input_string = ('\n'.join(matches['match_name'].dropna().astype(str)))

    company_name = match_company_name(input_string, df_english)

    input_score = ('\n'.join(matches['score'].dropna().astype(str)))
    input_score = float(input_score)
    if input_score >= 85:
        print(f"匹配到的公司名称是: {company_name}")
    else:
        print("没有匹配到相应的公司名称")

请输入匹配的名称:  网易(杭州)网络有限公司


preprocessing...

preprocessing complete 
 searching for matches...



100%|██████████| 1/1 [00:00<00:00, 122.58it/s]


possible matches found   
 fuzzy matching...



100%|██████████| 1/1 [00:00<00:00, 29.07it/s]

done
没有匹配到相应的公司名称


In [17]:
#Object-Oriented Programming Usage V5
import pandas as pd
import json
import re
import string
from deep_translator import GoogleTranslator
from name_matching.name_matcher import NameMatcher
from cleanco import basename
from fuzzychinese import FuzzyChineseMatch
from rapidfuzz import process

CJK_REGEX = re.compile(r'[\u4e00-\u9fff]')
CJK_OR_NUMERIC_REGEX = re.compile(r'(?P<cjk>[\u4E00-\u9FFF]+)|(?P<numeric>[0-9]+)')
ENGLISH_SPLITTER_REGEX = re.compile(r'\s+')

CIRCUMFLEX_PAIRS = \
    ((r'àâáãāåäǎ', 'a'), (r'ÀÂÁÃĀÅÄǍ', 'A'), (r'ß', 'b'), (r'čćç', 'c'), (r'ČÇ', 'C'), (r'ðđ', 'd'), (r'ÐĐ', 'D'),
     (r'éěèêëėệē', 'e'), (r'ÉÈÊËĖỆĒ', 'E'), (r'ğ', 'g'), (r'Ğ', 'G'), (r'îïíīịìǐĩı', 'i'), (r'İÎÏÍĪỊÌǏ', 'I'),
     (r'Ł', 'L'), (r'ł', 'l'), (r'ñňń', 'n'), (r'ÑŇŃ', 'N'), (r'ôöōồǒóòøőõ', 'o'), (r'ÔÖŌỒÓÒØŐÕ', 'O'), (r'ŕ', 'r'),
     (r'şš', 's'), (r'ŠŞ', 'S'), (r'ť', 't'), (r'ûùüúǔưū', 'u'), (r'ÛÙÜÚƯŪ', 'U'), (r'ŵ', 'w'), (r'ý', 'y'),
     (r'Ý', 'Y'), (r'žź', 'z'), (r'ŽŹ', 'Z'), (r'æ', 'ae'), (r'Æ', 'AE'))
CIRCUMFLEX_REGEX_PAIRS = tuple(
    (re.compile(rf'[{regex_}]'), format_) for (regex_, format_) in CIRCUMFLEX_PAIRS
)

CHINESE_PUNCTUATION = r'[！？｡＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏.]'


class DataProcessor:
    def __init__(self, input_path, output_path):
        self.input_path = input_path
        self.output_path = output_path

    def clean_chinese_strings(self, text):
        return re.sub(r'(?<=[\u4e00-\u9fff])\s+(?=[\u4e00-\u9fff])', '', text)

    def parse_and_clean_jsonl(self):
        cleaned_data = []
        with open(self.input_path, 'r', encoding='utf-8') as jsonl_file:
            for line in jsonl_file:
                data = json.loads(line)
                company_name = data.get('companyName', '')
                required_search_strings = data.get('requiredSearchStrings', [])
                aliases = data.get('aliases', [])

                cleaned_required_search_strings = [
                    self.clean_chinese_strings(string) if CJK_REGEX.search(string) else string
                    for string in required_search_strings
                ]
                cleaned_aliases = [
                    self.clean_chinese_strings(alias) if CJK_REGEX.search(alias) else alias
                    for alias in aliases
                ]

                cleaned_data.append({
                    'requiredSearchStrings': cleaned_required_search_strings,
                    'aliases': cleaned_aliases,
                    'companyName': company_name
                })

        with open(self.output_path, 'w', encoding='utf-8') as output_file:
            for item in cleaned_data:
                output_file.write(json.dumps(item, ensure_ascii=False) + '\n')

    def split_english_number_cjk(self, text, separate_return=True):
        en_number_words = []
        cjk_words = []
        start, end = 0, len(text)
        while start < end and (r_ := CJK_OR_NUMERIC_REGEX.search(text, pos=start)):
            if word := text[start:r_.start()].strip():
                en_number_words.append(word)
            if cjk_word := r_.groupdict().get("cjk"):
                cjk_words.append(cjk_word)
            else:
                numeric_word = r_.groupdict().get("numeric")
                en_number_words.append(numeric_word)
            start = r_.end()

        if end > start and (text := text[start:].strip()):
            en_number_words.append(text)
        return (en_number_words, cjk_words) if separate_return else en_number_words

    def parse_and_separate_jsonl(self):
        english_list = []
        chinese_list = []
        with open(self.output_path, 'r', encoding='utf-8') as jsonl_file:
            for line in jsonl_file:
                data = json.loads(line)
                company_name = data.get('companyName', '')
                required_search_strings = data.get('requiredSearchStrings', [])
                aliases = data.get('aliases', [])

                all_strings = list(set(required_search_strings + aliases))

                for string in all_strings:
                    en_strings, cjk_strings = self.split_english_number_cjk(string, separate_return=True)
                    for en_str in en_strings:
                        english_list.append({'Company Name': company_name, 'Search String': en_str})
                    for cjk_str in cjk_strings:
                        chinese_list.append({'Company Name': company_name, 'Search String': cjk_str})

        df_english = pd.DataFrame(english_list)
        df_chinese = pd.DataFrame(chinese_list)

        return df_english, df_chinese


class TextPreprocessor:
    def preprocess(self, text):
        text = text.lower()
        text = self.circumflex_regulator(text)
        text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
        text = basename(text)
        common_words = ['inc', 'ltd', 'corp', 'llc', 'co', 'company','group']
        for word in common_words:
            text = re.sub(r'\b{}\b'.format(re.escape(word)), '', text)
        text = ' '.join(text.split())
        return text

    def circumflex_regulator(self, name: str):
        if not name:
            return ''
        for regex_, format_character in CIRCUMFLEX_REGEX_PAIRS:
            name = regex_.sub(format_character, name)
        return name


class NameMatcherManager:
    def __init__(self):
        self.matcher = NameMatcher(number_of_matches=1,
                                   legal_suffixes=True,
                                   common_words=False,
                                   top_n=50,
                                   verbose=True)

    def load_and_process_data(self, df_english):
        self.matcher.load_and_process_master_data(column='Search String', df_matching_data=df_english, transform=True)

    def match_names(self, input_string, df_english):
        df_temp = pd.DataFrame([{'Search String': input_string}])
        matches = self.matcher.match_names(to_be_matched=df_temp, column_matching='Search String')
        input_string = ('\n'.join(matches['match_name'].dropna().astype(str)))
        company_name = self.match_company_name(input_string, df_english)
        input_score = ('\n'.join(matches['score'].dropna().astype(str)))
        input_score = float(input_score)
        return company_name, input_score

    def match_company_name(self, input_string, df):
        best_match = process.extractOne(input_string, df['Search String'])
        if best_match:
            matched_row = df[df['Search String'] == best_match[0]]
            return matched_row['Company Name'].values[0]
        else:
            return None


class FuzzyMatcher:
    def __init__(self, csv_file='chinese_search_strings.csv'):
        self.csv_file = csv_file
        self.fcm = FuzzyChineseMatch(ngram_range=(2, 3), analyzer='char')

    def find_company_name(self, user_input):
        df = pd.read_csv(self.csv_file)
        search_strings = df['Search String']
        company_names = df['Company Name']
        self.fcm.fit(search_strings)
        user_input_series = pd.Series([user_input])
        top_matches = self.fcm.transform(user_input_series, n=1)
        matched_index = self.fcm.get_index()[0][0]
        matched_company_name = company_names.iloc[matched_index]
        similarity_scores = self.fcm.get_similarity_score()
        top1_score = similarity_scores[0][0]
        return matched_company_name, top1_score


class CompanyNameMatcher:
    def __init__(self, input_file_path, output_file_path):
        self.data_processor = DataProcessor(input_file_path, output_file_path)
        self.text_preprocessor = TextPreprocessor()
        self.name_matcher_manager = NameMatcherManager()
        self.fuzzy_matcher = FuzzyMatcher()

    def run(self):
        # 解析和清理数据
        self.data_processor.parse_and_clean_jsonl()
        df_english, df_chinese = self.data_processor.parse_and_separate_jsonl()
        df_chinese.to_csv('chinese_search_strings.csv', index=False, encoding='utf-8-sig')
        df_english.to_csv('english_search_strings.csv', index=False, encoding='utf-8-sig')

        # 加载和处理数据
        self.name_matcher_manager.load_and_process_data(df_english)

        # 输入匹配
        user_input = input("请输入匹配的名称: ")
        if any(CJK_REGEX.match(char) for char in user_input):  # 包含中文字符
            user_input = re.sub(CHINESE_PUNCTUATION, '', user_input)
            if all(CJK_REGEX.match(char) for char in user_input):  # 全部是中文字符
                matched_company_name, top1_score = self.fuzzy_matcher.find_company_name(user_input)
                if top1_score >= 0.6:
                    print(f"匹配到的公司名称是: {matched_company_name}, 匹配得分是: {top1_score}")
                else:
                    print("没有匹配到相应的公司名称")
            else:  # 混合中文和英文字符，或包含其他外文字符
                translated_input = GoogleTranslator(source='auto', target='en').translate(user_input)
                processed_input = self.text_preprocessor.preprocess(translated_input)
                company_name, input_score = self.name_matcher_manager.match_names(processed_input, df_english)
                if input_score >= 85:
                    print(f"匹配到的公司名称是: {company_name}")
                else:
                    print("没有匹配到相应的公司名称")
        else:  # 全部是英文字符
            processed_input = self.text_preprocessor.preprocess(user_input)
            company_name, input_score = self.name_matcher_manager.match_names(processed_input, df_english)
            if input_score >= 85:
                print(f"匹配到的公司名称是: {company_name}")
            else:
                print("没有匹配到相应的公司名称")


# 执行匹配过程
if __name__ == "__main__":
    input_file_path = 'bd_companies_international.jsonl'
    output_file_path = 'cleaned_bd_companies_international.jsonl'
    
    # 创建CompanyNameMatcher实例
    matcher = CompanyNameMatcher(input_file_path, output_file_path)
    
    # 运行匹配过程
    matcher.run()

请输入匹配的名称:  HP Enterprise Services


preprocessing...

preprocessing complete 
 searching for matches...



100%|██████████| 1/1 [00:00<00:00, 115.21it/s]


possible matches found   
 fuzzy matching...



100%|██████████| 1/1 [00:00<00:00, 30.57it/s]

done
匹配到的公司名称是: BFG Enterprise Services


In [ ]:
#以下为只用Name Matching匹配，忽略FuzzyCHinese
#V6
import pandas as pd
import json
import jieba
from name_matching.name_matcher import NameMatcher

# 定义中文分词函数
def chinese_tokenizer(text):
    return " ".join(jieba.cut(text))

# 加载JSONL文件
def load_company_data(file_path):
    companies = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            companies.append(json.loads(line))
    return companies

# 将数据整理为DataFrame格式，便于name_matching处理
def prepare_matching_data(companies):
    rows = []
    for company in companies:
        for search_string in company.get("requiredSearchStrings", []):
            rows.append({"name": search_string, "companyName": company["companyName"]})
        for alias in company.get("aliases", []):
            rows.append({"name": alias, "companyName": company["companyName"]})
    return pd.DataFrame(rows)

# 匹配用户输入
def match_user_input(user_input, matching_data):
    matcher = NameMatcher(remove_ascii=False, punctuations=False)  # 保留中文和标点符号

    # 分词处理输入数据和匹配数据
    matching_data['name'] = matching_data['name'].apply(chinese_tokenizer)
    matcher.load_and_process_master_data(column="name", df_matching_data=matching_data)

    # 将用户输入进行分词处理
    user_input_processed = chinese_tokenizer(user_input)

    # 匹配用户输入
    to_be_matched = pd.DataFrame({"name": [user_input_processed]})
    result = matcher.match_names(to_be_matched, column_matching="name")

    if not result.empty:
        # 返回匹配到的公司名称
        best_match = result.iloc[0]["match_name"]  # 取第一个匹配结果
        company_name = matching_data[matching_data['name'] == best_match]["companyName"].values[0]
        return company_name
    return None

# 主函数
def main(user_input):
    # 加载公司数据
    companies = load_company_data('bd_companies_international.jsonl')
    
    # 准备数据供name_matching使用
    matching_data = prepare_matching_data(companies)
    
    # 匹配用户输入并返回公司名称
    company_name = match_user_input(user_input, matching_data)
    
    if company_name:
        print(f"匹配到的公司名称是: {company_name}")
    else:
        print("未匹配到相关公司")

# 用户输入
user_input = "samsung"
main(user_input)

In [ ]:
#V7
#Add a Match_Score
import pandas as pd
import json
import jieba
from name_matching.name_matcher import NameMatcher

# 定义中文分词函数
def chinese_tokenizer(text):
    return " ".join(jieba.cut(text))

# 加载JSONL文件
def load_company_data(file_path):
    companies = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            companies.append(json.loads(line))
    return companies

# 将数据整理为DataFrame格式，便于name_matching处理
def prepare_matching_data(companies):
    rows = []
    for company in companies:
        for search_string in company.get("requiredSearchStrings", []):
            rows.append({"name": search_string, "companyName": company["companyName"]})
        for alias in company.get("aliases", []):
            rows.append({"name": alias, "companyName": company["companyName"]})
    return pd.DataFrame(rows)

# 匹配用户输入
def match_user_input(user_input, matching_data):
    matcher = NameMatcher(remove_ascii=False, punctuations=False)  # 保留中文和标点符号

    # 分词处理输入数据和匹配数据
    matching_data['name'] = matching_data['name'].apply(chinese_tokenizer)
    matcher.load_and_process_master_data(column="name", df_matching_data=matching_data)

    # 将用户输入进行分词处理
    user_input_processed = chinese_tokenizer(user_input)

    # 匹配用户输入
    to_be_matched = pd.DataFrame({"name": [user_input_processed]})
    result = matcher.match_names(to_be_matched, column_matching="name")

    if not result.empty:
        # 获取第一个匹配结果的得分
        best_match = result.iloc[0]["match_name"]  # 取第一个匹配结果
        match_score = result.iloc[0]["score"]  # 获取第一个匹配的得分

        # 检查得分是否达到60%
        if match_score >= 60:
            company_name = matching_data[matching_data['name'] == best_match]["companyName"].values[0]
            return company_name
        else:
            return "未找到相关公司"  
    return "未找到相关公司"

# 主函数
def main(user_input):
    # 加载公司数据
    companies = load_company_data('bd_companies_international.jsonl')
    
    # 准备数据供name_matching使用
    matching_data = prepare_matching_data(companies)
    
    # 匹配用户输入并返回公司名称
    company_name = match_user_input(user_input, matching_data)
    
    print(company_name)

# 用户输入
user_input = input("Enter a company name:")
main(user_input)


In [ ]:
#V8
import re
import pandas as pd
import json
import jieba
from name_matching.name_matcher import NameMatcher
from functools import partial

AND_SYMBOL_REGEX = re.compile(r'[a-z0-9](&)[a-z0-9]', re.IGNORECASE)
AT_SYMBOL_REGEX = re.compile(r'[a-z0-9]+(@)[a-z0-9]+', re.IGNORECASE)

# CIRCUMFLEX_PAIRS Usage:
#    Remove circumflex from characters in search string
#    mapping: 大小写声调字母 -> 标准英文字母
CIRCUMFLEX_PAIRS = \
    ((r'àâáãāåäǎ', 'a'), (r'ÀÂÁÃĀÅÄǍ', 'A'), (r'ß', 'b'), (r'čćç', 'c'), (r'ČÇ', 'C'), (r'ðđ', 'd'), (r'ÐĐ', 'D'),
     (r'éěèêëėệē', 'e'), (r'ÉÈÊËĖỆĒ', 'E'), (r'ğ', 'g'), (r'Ğ', 'G'), (r'îïíīịìǐĩı', 'i'), (r'İÎÏÍĪỊÌǏ', 'I'),
     (r'Ł', 'L'), (r'ł', 'l'), (r'ñňń', 'n'), (r'ÑŇŃ', 'N'), (r'ôöōồǒóòøőõ', 'o'), (r'ÔÖŌỒÓÒØŐÕ', 'O'), (r'ŕ', 'r'),
     (r'şš', 's'), (r'ŠŞ', 'S'), (r'ť', 't'), (r'ûùüúǔưū', 'u'), (r'ÛÙÜÚƯŪ', 'U'), (r'ŵ', 'w'), (r'ý', 'y'),
     (r'Ý', 'Y'), (r'žź', 'z'), (r'ŽŹ', 'Z'), (r'æ', 'ae'), (r'Æ', 'AE'))
CIRCUMFLEX_REGEX_PAIRS = tuple(
    (re.compile(rf'[{regex_}]'), format_) for (regex_, format_) in CIRCUMFLEX_PAIRS
)

ENGLISH_SPLITTER_REGEX = re.compile(r"[^\w&_+*\\/'#\-]+")
CJK_CHARACTERS = r'\u1100-\u11ff\u2e80-\u2fff\u3040-\u31ff\u3400-\u9fff\ua960-\ua97f\uac00-\ud7ff\uf900-\ufaff'
CJK_OR_NUMERIC_REGEX = re.compile(rf"(?P<cjk>[{CJK_CHARACTERS}]+)|(?P<numeric>((?<=^\D)|(?<=[^\W0-9_]|\s))(?<!\b[a-zA-Z])(\d+([\W_]{{0,5}}\d+){{0,5}})(?=($|[^\W0-9_]|\s)))")

ENTITY_WORDS_SPLITTER = re.compile('[%s]+' % re.escape('!"%\'*<=>?@^_`~－ \t\n'))
# 保留 meaning word(\w) 数字 下划线
MEANINGLESS_COLLEGE_COMPANY_NAME_REGEX = re.compile(
    rf"[^0-9\w\s]|^其[他它她]$|^others?$|^n\.?a\.?$|^n/a$|^none$|^null$",
    re.IGNORECASE
)
WHITESPACE_REGEX = re.compile(r'\s+')
ERASE_OR_REPLACE_TO_SPACE_SYMBOLS = re.compile(r'''[/\\%!！.?？^*·•⦁\-]''')
REPLACE_TO_SPACE_SYMBOLS = re.compile(r'&?#.*|(\s+|^)[+&@](\s+|$)|\s*[_+()（）、,，–\-]\s*')
THE_SPECIAL_ENTITY_NAME_REGEX = re.compile(r'^(the\s+[a-z0-9]+)$', re.IGNORECASE)
SINGLE_LETTER_SEQUENTIAL_REGEX = re.compile(rf'(\b[a-z]\b)(\s+[a-z]\b)+', re.IGNORECASE)
SINGLE_LOWERCASE_REGEX = re.compile(rf'\s+[a-z]\s+')
REGEX_COMPANY_DROP_INVALID = re.compile(r'[~$()@—－（）√《》:;：；.、•·‧・，*?+&|,/\[\\]|{}\t\n"]|([-]+\s|\s[-]+)')

PREPOSITION_WORDS = {
    'aboard', 'about', 'above', 'across', 'after', 'against', 'along', 'amid', 'among', 'anti', 'around', 'as', 'at',
    'before', 'behind', 'below', 'beneath', 'beside', 'besides', 'between', 'beyond', 'but', 'by',
    'concerning', 'considering', 'despite', 'down', 'during',
    'except', 'excepting', 'excluding', 'following', 'for', 'from',
    'in', 'inside', 'into', 'like', 'minus', 'near',
    'of', 'off', 'on', 'onto', 'opposite', 'outside', 'over',
    'past', 'per', 'plus', 'regarding', 'round',
    'since', 'than', 'through', 'till', 'to', 'toward', 'towards',
    'under', 'underneath', 'unlike', 'until', 'up', 'upon',
    'versus', 'via', 'with', 'within', 'without'}

CONJUNCTIONS_WORDS = {
    "although", "because", "before", "both", "and", '&', "whether", "or", "either", "neither", "nor", "once",
    "just", "so", "as", "if", "then", "rather", "than", "till", 'when', 'where', 'whenever', 'while', 'wherever',
    "such", "so", "that"}

ARTICLES = {'a', 'an', 'the'}

INLINE_WORDS = {
                   'also', 'am', 'are', 'did', 'furthermore',
                   'has', 'hence', 'how', 'however', 'includ.',
                   'instead', 'is', 'likewise', 'long',
                   'moreover', 'should', 'similar',
                   'though', 'thus', 'unless',
                   'was', 'were', 'what', 'which', 'whichever', 'why', 'will',
                   # 葡萄牙语介词    https://sites.google.com/site/omundodo1/yu-fa/1-2
                   # para couldn't delete: "Federal University of Pará"
                   'de', 'di', 'em', 'del', 'des', 'do',
                   # 意大利语介词
                   'delle',
                   # 西班牙语介词    https://www.sohu.com/a/139510376_509791
                   # 'a,con,sobre, en,de,contra,desde,entre,hacia,por'
                   'con', 'sobre', 'en', 'contra', 'desde', 'entre', 'hacia', 'por', 'la',
                   # 法语介词  https://zhuanlan.zhihu.com/p/33125505
                   'apres', 'avant', 'avec', 'chez', 'contre', 'dans', 'depuis', 'derriere', 'devant', 'durant', 'envers', 'environ',
                   'jusque', 'malgre', 'par', 'parmi', 'pendant', 'pour', 'sans', 'selon', 'sous', 'suivant', 'sur', 'vers'
               } | PREPOSITION_WORDS | CONJUNCTIONS_WORDS | ARTICLES

IMPOSSIBLE_ENDING_WORDS = INLINE_WORDS | {
    'I', 'on', 'in', 'for',  # in jd and resume, it normally inside sentences
    'the', 'my', 'your', 'he', 'she', 'his', 'her', 'fewer'
}
# 规范化罗马数字
ROMAN_NUM_REGULATOR = {'I': '1', 'II': '2', 'III': '3', 'IV': '4', 'iv': '4', 'V': '5'}

COMMON_WORDS = IMPOSSIBLE_ENDING_WORDS | \
               {'ago', 'on', 'consequently', 'behind', 'otherwise', 'the',
                'plus', 'whereas', 'out', 'whenever', 'above', 'soon', 'before',
                'around', 'neither', 'lest', 'down', 'both',
                'now', 'even', 'because', 'still', 'along', 'whatever', 'although', 'following', 'up', 'for', 'nor',
                'under', 'concerning', 'between', 'rather', 'towards', 'besides', 'either',
                'nonetheless', 'yet', 'much', 'off', 'in', 'for',
                'despite', 'once', 'beyond', 'therefore',
                'meanwhile', 'within', 'across', 'conversely', 'without', 'supposing', 'like', 'or', 'including',
                'throughout', 'wherever', 'nevertheless', 'accordingly', 'only', 'background', 'skill', 'backgrounds', 'skills'}
COMPANY_COMMON_WORDS = {
    "en": {
        "laboratory", "laboratories", "lab", "labs", "research", "technology", "technologies", "technical", "tech", "sci", "science"
    },
    "zh": {
        "实验室", "研究院", "技术", "科技"
    }
}
COMPANY_COMMON_WORDS_REGEX = re.compile(
    r"{0}|{1}".format(
        '|'.join(COMPANY_COMMON_WORDS['zh']),
        rf"(\b|^)({'|'.join(COMPANY_COMMON_WORDS['en'])})(\.|\b|$)"
    ), re.I)


def circumflex_regulator(name: str):
    if not name:
        return ''
    for regex_, format_character in CIRCUMFLEX_REGEX_PAIRS:
        name = regex_.sub(format_character, name)
    return name


def split_english_number_cjk(text: str, separate_return=False, split_same_language=False):
    """
    Chinese and Chinese-English Mixture Operations
    :param text: e.g. "non_asians string: 并删除掉Machine-learning多余的, 字符串"
    :param separate_return: False
    :param split_same_language: False
    :return: e.g. ["non_asians string:", "并删除掉", "Machine-learning", "多余的, 字符串"]
    :param separate_return: False
    :param split_same_language: True
    :return: e.g. ['non_asians', 'string', '并', '删', '除', '掉', 'Machine-learning', '多', '余', '的', '字', '符', '串']
    :param separate_return: True
    :param split_same_language: False
    :return: e.g. (["non_asians string:", "Machine-learning"], ["并删除掉", "多余的, 字符串"])
    :param separate_return: True
    :param split_same_language: True
    :return: e.g. (['non_asians', 'string', 'Machine-learning'], ['并', '删', '除', '掉', '多', '余', '的', '字', '符', '串'])
    """
    en_number_words = []
    cjk_words = []
    start, end = 0, len(text)
    while start < end and (r_ := CJK_OR_NUMERIC_REGEX.search(text, pos=start)):
        # before word
        if word := text[start:r_.start()].strip():
            if split_same_language:
                en_number_words.extend(
                    [stripped_word for word_ in ENGLISH_SPLITTER_REGEX.split(word) if (stripped_word := word_.strip())])
            else:
                en_number_words.append(word)
        # matched word
        if cjk_word := r_.groupdict().get("cjk"):
            if split_same_language:
                (cjk_words if separate_return else en_number_words).extend(list(cjk_word))
            else:
                (cjk_words if separate_return else en_number_words).append(cjk_word)
        else:
            numeric_word = r_.groupdict().get("numeric")
            if split_same_language:
                en_number_words.extend([stripped_word for word_ in ENGLISH_SPLITTER_REGEX.split(numeric_word) if (stripped_word := word_.strip())])
            else:
                en_number_words.append(numeric_word)
        start = r_.end()
    # save after left words
    if end > start and (text := text[start:].strip()):
        if split_same_language:
            en_number_words.extend([word_ for word in ENGLISH_SPLITTER_REGEX.split(text) if (word_ := word.strip())])
        else:
            en_number_words.append(text)
    return (en_number_words, cjk_words) if separate_return else en_number_words


def match_language_by_character_regex(text: str, regex):
    if text is None:
        return False
    else:
        return regex.search(text)


def search_string_regulator(name: str, return_str=True, is_company_name=False):
    """
    :param is_company_name: company name 不应去除重复单词以及停止词，例如有公司名叫 IN-CO, ANOTHER CO
    :param name:
        The input string will be regulated with the following requirements:
            1. Only Chinese/English/Digits characters left (remove characters in other languages)
            3. Segmented:
                Chinese: jieba cut words
                English: words separated by space
            3. Only Consists of meaningful words
    :param return_str:
    :return:
    """
    extracted_name = MEANINGLESS_COLLEGE_COMPANY_NAME_REGEX.sub('', name)
    #  判断是否仅有“无、其他它她等”
    if not extracted_name:
        return '' if return_str else []
    words = []
    for segment in ENTITY_WORDS_SPLITTER.split(extracted_name):
        if not segment:
            pass
        # chinese: split words
        elif contains_chinese(segment):
            words.extend([word_ for word_ in jieba.cut(segment)])
        # english: remove duplicates and common words
        elif number := ROMAN_NUM_REGULATOR.get(segment, None):
            words.append(number)
        elif (segment := segment.lower()) not in words or is_company_name:
            words.append(segment)
    if len(words) > 2 and not is_company_name:
        words = [word for word in words if word not in COMMON_WORDS]
    return ' '.join(words) if return_str else words


def regulate_english_asians_mixed_string(text: str):
    return ' '.join(split_english_number_cjk(text, separate_return=False))


# 必须同时出现在 query和cache中的 必要词
# 去除公司后缀等词后，仅剩如下词汇，则不可去除公司后缀
COMPANY_KEYWORDS_REGEX = re.compile(
    r'\b(bank|college|university|education|energy|finance|army|air foce|navy|industry|'
    r'hospital|hotel|institutes?|institution|petroleum|oil|health|electronic|commercial|'
    r'environmental|gover(?:nor?|ment)|state|estado|procter|reliance|genome|software|hardware|agency|'
    r'(?:sou|nor)th(?:[-\s]?(?:ea|we)st(?:ern)?)?|[东西南北]方|[东西][南北][方]?|环境|'
    r'银行|软件|硬件|大学|机构|研究院|学院|教育|协会|能源|金融|旅馆|医院|石油|电子|工业|[陆海空]军|政府|基因|'
    r'global|market(?:ing)?|real estate|system|房地产|系统|兴业)\b',
    re.IGNORECASE)
# Common words in organization names
WORLD_BOURSE = {'en': ('Euronext', 'NYSE', 'NYX', 'ICE', 'TSE', 'NASDAQ', 'HKEx', 'TSX', 'DBAG', 'OSE', 'SWX', 'SSE',
                       'SZSE', 'TSE', 'KRX', 'KOSDAQ', 'LSE', 'FWB', 'SGX', 'BSE', r'Dual[-\s]listed'),
                'zh': ('港交所', '纳斯达克', '那斯達克', '深交所', '上交所', '纽交所', r'[雙|双]重上市')}
REGEX_BOURSE = re.compile(r"({0}|{1})".
                          format('|'.join([rf'{suf}' for suf in WORLD_BOURSE['zh']]),
                                 '|'.join([rf'(?:^|\s){suf}(?:\.|\s|\:|$)' for suf in WORLD_BOURSE['en']])),
                          re.IGNORECASE)

ORG_SUFFIX_WORDS = {
    'zh': {'common': ('公司', '集团', '有限公司', '有限责任公司', '股份有限公司'),
           'uncommon': ('总公司', '股份', '控股', '责任', '有限', '企业', '协会', '合作社', '株式会社')},
    'en': {'common': ('company', 'group', 'corporation', 'incorporated', 'enterprise', 'enterprises',
                      'co', 'inc', 'corp', 'ltd', 'llc', 'se', 'pvt'),
           # drop 'shop' company suffix: 'Future Shop'/'Future plc'
           'uncommon': ('corporation limited', 'companies', 'worldwide', 'limited', "holding", "holdings",
                        'com', 'gmbh', 'ag', 'plc', 'sal', 'spa', r's\.p\.a', 'sab cv', 'sa', 'nv', r'n\.v', 'lp', 'sro',
                        'kg', 'aktiengesellschaft', 'de cv', 'ltda', "group of companies")}
}
# Common words regexp in organization names
REGEX_COMPANY_SUFFIX = re.compile(
    r"{0}|{1}".format(
        '|'.join([rf'{suf}' for suf in ORG_SUFFIX_WORDS['zh']['uncommon'] + ORG_SUFFIX_WORDS['zh']['common']]),
        rf"(?:\b(?:{'|'.join([suf for suf in ORG_SUFFIX_WORDS['en']['uncommon'] + ORG_SUFFIX_WORDS['en']['common']])})(?:\.|$))"
    ),
    re.IGNORECASE
)
CHINESE_CHARACTERS = r'\u2e80-\u2fff\u31c0-\u31ef\u3400-\u9fff\uf900-\ufaff'
contains_chinese = partial(match_language_by_character_regex, regex=re.compile(rf'[{CHINESE_CHARACTERS}]'))

URL_REGEX = re.compile(r"(?:http://|https://)?(?:www\.)?([a-z0-9]+(?:[\-.][a-z0-9]+)*)")
MULTI_TITLE_OR_REGEX = re.compile(r'(\w+(?:[\W]\w+)+)[/\\](?:\w+(?:\W\w+)+)')
SPLIT_OR_SYMBOLS = re.compile(r'\s[/\\\-]\s')
OR_AS_SYMBOL_REPLACE_REGEX = re.compile(r"^[a-z0-9]+[/\\]([a-z0-9]+$|[a-z0-9]\b)", re.IGNORECASE)

REGEX_EN = re.compile(f"[a-zA-Z]+")



def company_words_regulator(company_name: str):
    search_str = circumflex_regulator(company_name)

    if contains_chinese(search_str):
        if not REGEX_EN.search(search_str):
            search_str = search_str.replace(' ', '')

    search_str_drop = WHITESPACE_REGEX.sub(' ', REGEX_COMPANY_SUFFIX.sub('', REGEX_COMPANY_SUFFIX.sub('', search_str).strip())).strip()

    search_str_drop_bourse = REGEX_BOURSE.sub(' ', search_str_drop).strip()
    if search_str_drop_bourse:
        search_str_drop = search_str_drop_bourse

    search_str_drop_common_word = WHITESPACE_REGEX.sub(' ', COMPANY_COMMON_WORDS_REGEX.sub('', search_str_drop)).strip()
    if search_str_drop_common_word:
        search_str_drop = search_str_drop_common_word

    # 检查是否需要恢复原始字符串
    if (r_ := COMPANY_KEYWORDS_REGEX.search(search_str_drop)) and r_.end() - r_.start() > len(search_str_drop) - 2:
        pass
    elif search_str == search_str_drop:
        pass
    else:
        search_str = search_str_drop

    search_str_regulate = search_string_regulator(search_str, is_company_name=True).lower()
    if search_str_regulate != search_str:
        search_str = search_str_regulate

    valid_words = []
    for word_ in search_str.split(' '):
        word_ = REGEX_COMPANY_DROP_INVALID.sub(' ', word_).strip().lower()
        if word_:
            valid_words.append(word_)
    return ' '.join(valid_words)


# 定义中文分词函数
def chinese_tokenizer(text):
    return " ".join(jieba.cut(text))

# 加载JSONL文件
def load_company_data(file_path):
    companies = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            companies.append(json.loads(line))
    return companies

# 将数据整理为DataFrame格式，便于name_matching处理
def prepare_matching_data(companies):
    rows = []
    for company in companies:
        for search_string in company.get("requiredSearchStrings", []):
            rows.append({"name": search_string, "companyName": company["companyName"]})
        for alias in company.get("aliases", []):
            rows.append({"name": alias, "companyName": company["companyName"]})
    return pd.DataFrame(rows)

# 匹配用户输入
def match_user_input(user_input, matching_data):
    matcher = NameMatcher(remove_ascii=False, punctuations=False)  # 保留中文和标点符号

    # 分词处理输入数据和匹配数据
    matching_data['name'] = matching_data['name'].apply(chinese_tokenizer)
    matcher.load_and_process_master_data(column="name", df_matching_data=matching_data)
    

    # 匹配用户输入
    to_be_matched = pd.DataFrame({"name": [user_input]})
    result = matcher.match_names(to_be_matched, column_matching="name")

    if not result.empty:
        # 获取第一个匹配结果的得分
        best_match = result.iloc[0]["match_name"]  # 取第一个匹配结果
        match_score = result.iloc[0]["score"]  # 获取第一个匹配的得分

        # 检查得分是否达到60%
        if match_score >= 60:
            company_name = matching_data[matching_data['name'] == best_match]["companyName"].values[0]
            return company_name
        else:
            return "未找到相关公司"  
    return "未找到相关公司"

# 主函数
def main(user_input):
    # 加载公司数据
    companies = load_company_data('bd_companies_international.jsonl')
    
    # 使用regulator
    user_input = company_words_regulator(user_input)
    
    # 准备数据供name_matching使用
    matching_data = prepare_matching_data(companies)
    
    # 匹配用户输入并返回公司名称
    company_name = match_user_input(user_input, matching_data)
    
    print(company_name)

# 用户输入
user_input = input("Enter a company name:")
main(user_input)

In [ ]:
# V8 OOP version only wroks in jupyterlab
import re
import pandas as pd
import json
import jieba
from name_matching.name_matcher import NameMatcher

class CompanyNameProcessor:
    AND_SYMBOL_REGEX = re.compile(r'[a-z0-9](&)[a-z0-9]', re.IGNORECASE)
    AT_SYMBOL_REGEX = re.compile(r'[a-z0-9]+(@)[a-z0-9]+', re.IGNORECASE)

    # CIRCUMFLEX_PAIRS Usage:
    #    Remove circumflex from characters in search string
    #    mapping: 大小写声调字母 -> 标准英文字母
    CIRCUMFLEX_PAIRS = \
        ((r'àâáãāåäǎ', 'a'), (r'ÀÂÁÃĀÅÄǍ', 'A'), (r'ß', 'b'), (r'čćç', 'c'), (r'ČÇ', 'C'), (r'ðđ', 'd'), (r'ÐĐ', 'D'),
         (r'éěèêëėệē', 'e'), (r'ÉÈÊËĖỆĒ', 'E'), (r'ğ', 'g'), (r'Ğ', 'G'), (r'îïíīịìǐĩı', 'i'), (r'İÎÏÍĪỊÌǏ', 'I'),
         (r'Ł', 'L'), (r'ł', 'l'), (r'ñňń', 'n'), (r'ÑŇŃ', 'N'), (r'ôöōồǒóòøőõ', 'o'), (r'ÔÖŌỒÓÒØŐÕ', 'O'), (r'ŕ', 'r'),
         (r'şš', 's'), (r'ŠŞ', 'S'), (r'ť', 't'), (r'ûùüúǔưū', 'u'), (r'ÛÙÜÚƯŪ', 'U'), (r'ŵ', 'w'), (r'ý', 'y'),
         (r'Ý', 'Y'), (r'žź', 'z'), (r'ŽŹ', 'Z'), (r'æ', 'ae'), (r'Æ', 'AE'))
    CIRCUMFLEX_REGEX_PAIRS = tuple(
        (re.compile(rf'[{regex_}]'), format_) for (regex_, format_) in CIRCUMFLEX_PAIRS
    )

    ENGLISH_SPLITTER_REGEX = re.compile(r"[^\w&_+*\\/'#\-]+")
    CJK_CHARACTERS = r'\u1100-\u11ff\u2e80-\u2fff\u3040-\u31ff\u3400-\u9fff\ua960-\ua97f\uac00-\ud7ff\uf900-\ufaff'
    CJK_OR_NUMERIC_REGEX = re.compile(rf"(?P<cjk>[{CJK_CHARACTERS}]+)|(?P<numeric>((?<=^\D)|(?<=[^\W0-9_]|\s))(?<!\b[a-zA-Z])(\d+([\W_]{{0,5}}\d+){{0,5}})(?=($|[^\W0-9_]|\s)))")

    ENTITY_WORDS_SPLITTER = re.compile('[%s]+' % re.escape('!"%\'*<=>?@^_`~－ \t\n'))
    # 保留 meaning word(\w) 数字 下划线
    MEANINGLESS_COLLEGE_COMPANY_NAME_REGEX = re.compile(
        rf"[^0-9\w\s]|^其[他它她]$|^others?$|^n\.?a\.?$|^n/a$|^none$|^null$",
        re.IGNORECASE
    )
    WHITESPACE_REGEX = re.compile(r'\s+')
    ERASE_OR_REPLACE_TO_SPACE_SYMBOLS = re.compile(r'''[/\\%!！.?？^*·•⦁\-]''')
    REPLACE_TO_SPACE_SYMBOLS = re.compile(r'&?#.*|(\s+|^)[+&@](\s+|$)|\s*[_+()（）、,，–\-]\s*')
    THE_SPECIAL_ENTITY_NAME_REGEX = re.compile(r'^(the\s+[a-z0-9]+)$', re.IGNORECASE)
    SINGLE_LETTER_SEQUENTIAL_REGEX = re.compile(rf'(\b[a-z]\b)(\s+[a-z]\b)+', re.IGNORECASE)
    SINGLE_LOWERCASE_REGEX = re.compile(rf'\s+[a-z]\s+')
    REGEX_COMPANY_DROP_INVALID = re.compile(r'[~$()@—－（）√《》:;：；.、•·‧・，*?+&|,/\[\\]|{}\t\n"]|([-]+\s|\s[-]+)')

    PREPOSITION_WORDS = {
        'aboard', 'about', 'above', 'across', 'after', 'against', 'along', 'amid', 'among', 'anti', 'around', 'as', 'at',
        'before', 'behind', 'below', 'beneath', 'beside', 'besides', 'between', 'beyond', 'but', 'by',
        'concerning', 'considering', 'despite', 'down', 'during',
        'except', 'excepting', 'excluding', 'following', 'for', 'from',
        'in', 'inside', 'into', 'like', 'minus', 'near',
        'of', 'off', 'on', 'onto', 'opposite', 'outside', 'over',
        'past', 'per', 'plus', 'regarding', 'round',
        'since', 'than', 'through', 'till', 'to', 'toward', 'towards',
        'under', 'underneath', 'unlike', 'until', 'up', 'upon',
        'versus', 'via', 'with', 'within', 'without'}

    CONJUNCTIONS_WORDS = {
        "although", "because", "before", "both", "and", '&', "whether", "or", "either", "neither", "nor", "once",
        "just", "so", "as", "if", "then", "rather", "than", "till", 'when', 'where', 'whenever', 'while', 'wherever',
        "such", "so", "that"}

    ARTICLES = {'a', 'an', 'the'}

    INLINE_WORDS = {
                       'also', 'am', 'are', 'did', 'furthermore',
                       'has', 'hence', 'how', 'however', 'includ.',
                       'instead', 'is', 'likewise', 'long',
                       'moreover', 'should', 'similar',
                       'though', 'thus', 'unless',
                       'was', 'were', 'what', 'which', 'whichever', 'why', 'will',
                       # 葡萄牙语介词    https://sites.google.com/site/omundodo1/yu-fa/1-2
                       # para couldn't delete: "Federal University of Pará"
                       'de', 'di', 'em', 'del', 'des', 'do',
                       # 意大利语介词
                       'delle',
                       # 西班牙语介词    https://www.sohu.com/a/139510376_509791
                       # 'a,con,sobre, en,de,contra,desde,entre,hacia,por'
                       'con', 'sobre', 'en', 'contra', 'desde', 'entre', 'hacia', 'por', 'la',
                       # 法语介词  https://zhuanlan.zhihu.com/p/33125505
                       'apres', 'avant', 'avec', 'chez', 'contre', 'dans', 'depuis', 'derriere', 'devant', 'durant', 'envers', 'environ',
                       'jusque', 'malgre', 'par', 'parmi', 'pendant', 'pour', 'sans', 'selon', 'sous', 'suivant', 'sur', 'vers'
                   } | PREPOSITION_WORDS | CONJUNCTIONS_WORDS | ARTICLES

    IMPOSSIBLE_ENDING_WORDS = INLINE_WORDS | {
        'I', 'on', 'in', 'for',  # in jd and resume, it normally inside sentences
        'the', 'my', 'your', 'he', 'she', 'his', 'her', 'fewer'
    }
    # 规范化罗马数字
    ROMAN_NUM_REGULATOR = {'I': '1', 'II': '2', 'III': '3', 'IV': '4', 'iv': '4', 'V': '5'}

    COMMON_WORDS = IMPOSSIBLE_ENDING_WORDS | \
                   {'ago', 'on', 'consequently', 'behind', 'otherwise', 'the',
                    'plus', 'whereas', 'out', 'whenever', 'above', 'soon', 'before',
                    'around', 'neither', 'lest', 'down', 'both',
                    'now', 'even', 'because', 'still', 'along', 'whatever', 'although', 'following', 'up', 'for', 'nor',
                    'under', 'concerning', 'between', 'rather', 'towards', 'besides', 'either',
                    'nonetheless', 'yet', 'much', 'off', 'in', 'for',
                    'despite', 'once', 'beyond', 'therefore',
                    'meanwhile', 'within', 'across', 'conversely', 'without', 'supposing', 'like', 'or', 'including',
                    'throughout', 'wherever', 'nevertheless', 'accordingly', 'only', 'background', 'skill', 'backgrounds', 'skills'}
    COMPANY_COMMON_WORDS = {
        "en": {
            "laboratory", "laboratories", "lab", "labs", "research", "technology", "technologies", "technical", "tech", "sci", "science"
        },
        "zh": {
            "实验室", "研究院", "技术", "科技"
        }
    }
    COMPANY_COMMON_WORDS_REGEX = re.compile(
        r"{0}|{1}".format(
            '|'.join(COMPANY_COMMON_WORDS['zh']),
            rf"(\b|^)({'|'.join(COMPANY_COMMON_WORDS['en'])})(\.|\b|$)"
        ), re.I)
    # 必须同时出现在 query和cache中的 必要词
    # 去除公司后缀等词后，仅剩如下词汇，则不可去除公司后缀
    COMPANY_KEYWORDS_REGEX = re.compile(
        r'\b(bank|college|university|education|energy|finance|army|air foce|navy|industry|'
        r'hospital|hotel|institutes?|institution|petroleum|oil|health|electronic|commercial|'
        r'environmental|gover(?:nor?|ment)|state|estado|procter|reliance|genome|software|hardware|agency|'
        r'(?:sou|nor)th(?:[-\s]?(?:ea|we)st(?:ern)?)?|[东西南北]方|[东西][南北][方]?|环境|'
        r'银行|软件|硬件|大学|机构|研究院|学院|教育|协会|能源|金融|旅馆|医院|石油|电子|工业|[陆海空]军|政府|基因|'
        r'global|market(?:ing)?|real estate|system|房地产|系统|兴业)\b',
        re.IGNORECASE)
    # Common words in organization names
    WORLD_BOURSE = {'en': ('Euronext', 'NYSE', 'NYX', 'ICE', 'TSE', 'NASDAQ', 'HKEx', 'TSX', 'DBAG', 'OSE', 'SWX', 'SSE',
                           'SZSE', 'TSE', 'KRX', 'KOSDAQ', 'LSE', 'FWB', 'SGX', 'BSE', r'Dual[-\s]listed'),
                    'zh': ('港交所', '纳斯达克', '那斯達克', '深交所', '上交所', '纽交所', r'[雙|双]重上市')}
    REGEX_BOURSE = re.compile(r"({0}|{1})".
                              format('|'.join([rf'{suf}' for suf in WORLD_BOURSE['zh']]),
                                     '|'.join([rf'(?:^|\s){suf}(?:\.|\s|\:|$)' for suf in WORLD_BOURSE['en']])),
                              re.IGNORECASE)
    
    ORG_SUFFIX_WORDS = {
        'zh': {'common': ('公司', '集团', '有限公司', '有限责任公司', '股份有限公司'),
               'uncommon': ('总公司', '股份', '控股', '责任', '有限', '企业', '协会', '合作社', '株式会社')},
        'en': {'common': ('company', 'group', 'corporation', 'incorporated', 'enterprise', 'enterprises',
                          'co', 'inc', 'corp', 'ltd', 'llc', 'se', 'pvt'),
               # drop 'shop' company suffix: 'Future Shop'/'Future plc'
               'uncommon': ('corporation limited', 'companies', 'worldwide', 'limited', "holding", "holdings",
                            'com', 'gmbh', 'ag', 'plc', 'sal', 'spa', r's\.p\.a', 'sab cv', 'sa', 'nv', r'n\.v', 'lp', 'sro',
                            'kg', 'aktiengesellschaft', 'de cv', 'ltda', "group of companies")}
    }
    # Common words regexp in organization names
    REGEX_COMPANY_SUFFIX = re.compile(
        r"{0}|{1}".format(
            '|'.join([rf'{suf}' for suf in ORG_SUFFIX_WORDS['zh']['uncommon'] + ORG_SUFFIX_WORDS['zh']['common']]),
            rf"(?:\b(?:{'|'.join([suf for suf in ORG_SUFFIX_WORDS['en']['uncommon'] + ORG_SUFFIX_WORDS['en']['common']])})(?:\.|$))"
        ),
        re.IGNORECASE
    )
    CHINESE_CHARACTERS = r'\u2e80-\u2fff\u31c0-\u31ef\u3400-\u9fff\uf900-\ufaff'
    contains_chinese = partial(match_language_by_character_regex, regex=re.compile(rf'[{CHINESE_CHARACTERS}]'))
    
    URL_REGEX = re.compile(r"(?:http://|https://)?(?:www\.)?([a-z0-9]+(?:[\-.][a-z0-9]+)*)")
    MULTI_TITLE_OR_REGEX = re.compile(r'(\w+(?:[\W]\w+)+)[/\\](?:\w+(?:\W\w+)+)')
    SPLIT_OR_SYMBOLS = re.compile(r'\s[/\\\-]\s')
    OR_AS_SYMBOL_REPLACE_REGEX = re.compile(r"^[a-z0-9]+[/\\]([a-z0-9]+$|[a-z0-9]\b)", re.IGNORECASE)

    REGEX_EN = re.compile(f"[a-zA-Z]+")


    def __init__(self, file_path):
        self.file_path = file_path
        self.matcher = NameMatcher(remove_ascii=False, punctuations=False)

    def load_company_data(self):
        companies = []
        with open(self.file_path, 'r', encoding='utf-8') as f:
            for line in f:
                companies.append(json.loads(line))
        return companies

    def prepare_matching_data(self, companies):
        rows = []
        for company in companies:
            for search_string in company.get("requiredSearchStrings", []):
                rows.append({"name": search_string, "companyName": company["companyName"]})
            for alias in company.get("aliases", []):
                rows.append({"name": alias, "companyName": company["companyName"]})
        return pd.DataFrame(rows)

    def circumflex_regulator(self, name):
        if not name:
            return ''
        for regex_, format_character in CIRCUMFLEX_REGEX_PAIRS:
            name = regex_.sub(format_character, name)
        return name

    def split_english_number_cjk(self, text, separate_return=False, split_same_language=False):
        en_number_words = []
        cjk_words = []
        start, end = 0, len(text)
        while start < end and (r_ := CJK_OR_NUMERIC_REGEX.search(text, pos=start)):
            if word := text[start:r_.start()].strip():
                if split_same_language:
                    en_number_words.extend([stripped_word for word_ in ENGLISH_SPLITTER_REGEX.split(word) if (stripped_word := word_.strip())])
                else:
                    en_number_words.append(word)
            if cjk_word := r_.groupdict().get("cjk"):
                if split_same_language:
                    (cjk_words if separate_return else en_number_words).extend(list(cjk_word))
                else:
                    (cjk_words if separate_return else en_number_words).append(cjk_word)
            else:
                numeric_word = r_.groupdict().get("numeric")
                if split_same_language:
                    en_number_words.extend([stripped_word for word_ in ENGLISH_SPLITTER_REGEX.split(numeric_word) if (stripped_word := word_.strip())])
                else:
                    en_number_words.append(numeric_word)
            start = r_.end()
        if end > start and (text := text[start:].strip()):
            if split_same_language:
                en_number_words.extend([word_ for word in ENGLISH_SPLITTER_REGEX.split(text) if (word_ := word.strip())])
            else:
                en_number_words.append(text)
        return (en_number_words, cjk_words) if separate_return else en_number_words

    def match_language_by_character_regex(self, text, regex):
        if text is None:
            return False
        else:
            return bool(regex.search(text))

    def search_string_regulator(self, name, return_str=True, is_company_name=False):
        extracted_name = MEANINGLESS_COLLEGE_COMPANY_NAME_REGEX.sub('', name)
        if not extracted_name:
            return '' if return_str else []
        words = []
        for segment in ENTITY_WORDS_SPLITTER.split(extracted_name):
            if not segment:
                continue
            elif contains_chinese(segment):
                words.extend([word_ for word_ in jieba.cut(segment)])
            elif number := ROMAN_NUM_REGULATOR.get(segment, None):
                words.append(number)
            elif (segment := segment.lower()) not in words or is_company_name:
                words.append(segment)
        if len(words) > 2 and not is_company_name:
            words = [word for word in words if word not in COMMON_WORDS]
        return ' '.join(words) if return_str else words

    def company_words_regulator(self, company_name: str) -> str:
        # 处理上标字符
        search_str = self.circumflex_regulator(company_name)

        # 删除常见公司后缀
        search_str = self.REGEX_COMPANY_SUFFIX.sub('', search_str).strip()

        # 删除额外空白字符
        search_str = self.WHITESPACE_REGEX.sub(' ', search_str).strip()

        # 特定词语的处理，如证券、股票等
        search_str = self.REGEX_BOURSE.sub(' ', search_str).strip()

        # 删除公司常见无意义词语
        search_str = self.COMPANY_COMMON_WORDS_REGEX.sub('', search_str).strip()

        # 如果字符串较短或匹配特定关键字，可能不需要进一步处理
        if self.COMPANY_KEYWORDS_REGEX.search(search_str) and len(search_str) < 5:
            return search_str

        # 对字符串进行进一步规范化处理，如分词等
        regulated_search_str = self.search_string_regulator(search_str, is_company_name=True)
        
        # 修正为小写
        regulated_search_str = regulated_search_str.lower()

        # 分词后保留有效单词
        valid_words = []
        for word in regulated_search_str.split():
            cleaned_word = self.ENGLISH_SPLITTER_REGEX.sub(' ', word).strip()
            if cleaned_word:
                valid_words.append(cleaned_word)

        # 返回处理后的公司名称
        return ' '.join(valid_words)

    def regulate_english_asians_mixed_string(self, text):
        return ' '.join(self.split_english_number_cjk(text, separate_return=False))

    def match_user_input(self, user_input, matching_data):
        matching_data['name'] = matching_data['name'].apply(self.chinese_tokenizer)
        self.matcher.load_and_process_master_data(column="name", df_matching_data=matching_data)
        to_be_matched = pd.DataFrame({"name": [user_input]})
        result = self.matcher.match_names(to_be_matched, column_matching="name")

        if not result.empty:
            best_match = result.iloc[0]["match_name"]
            match_score = result.iloc[0]["score"]
            if match_score >= 60:
                company_name = matching_data[matching_data['name'] == best_match]["companyName"].values[0]
                return company_name
            else:
                return "未找到相关公司"
        return "未找到相关公司"

    def chinese_tokenizer(self, text):
        return " ".join(jieba.cut(text))

    def process_input(self, user_input):
        companies = self.load_company_data()
        user_input = self.regulate_english_asians_mixed_string(user_input)
        matching_data = self.prepare_matching_data(companies)
        company_name = self.match_user_input(user_input, matching_data)
        return company_name


def main():
    processor = CompanyNameProcessor('bd_companies_international.jsonl')
    user_input = input("Enter a company name: ")
    company_name = processor.process_input(user_input)
    print(company_name)

if __name__ == "__main__":
    main()


In [ ]:
#V9
#One that works well but no aliases adaptataion
import re
import pandas as pd
import json
import jieba
from name_matching.name_matcher import NameMatcher


class NameNormalizer:
    CIRCUMFLEX_PAIRS = (
        (r'àâáãāåäǎ', 'a'), (r'ÀÂÁÃĀÅÄǍ', 'A'), (r'ß', 'b'), (r'čćç', 'c'), (r'ČÇ', 'C'),
        (r'ðđ', 'd'), (r'ÐĐ', 'D'), (r'éěèêëėệē', 'e'), (r'ÉÈÊËĖỆĒ', 'E'), (r'ğ', 'g'),
        (r'Ğ', 'G'), (r'îïíīịìǐĩı', 'i'), (r'İÎÏÍĪỊÌǏ', 'I'), (r'Ł', 'L'), (r'ł', 'l'),
        (r'ñňń', 'n'), (r'ÑŇŃ', 'N'), (r'ôöōồǒóòøőõ', 'o'), (r'ÔÖŌỒÓÒØŐÕ', 'O'), (r'ŕ', 'r'),
        (r'şš', 's'), (r'ŠŞ', 'S'), (r'ť', 't'), (r'ûùüúǔưū', 'u'), (r'ÛÙÜÚƯŪ', 'U'),
        (r'ŵ', 'w'), (r'ý', 'y'), (r'Ý', 'Y'), (r'žź', 'z'), (r'ŽŹ', 'Z'), (r'æ', 'ae'), (r'Æ', 'AE')
    )
    CIRCUMFLEX_REGEX_PAIRS = tuple(
        (re.compile(rf'[{regex_}]'), format_) for (regex_, format_) in CIRCUMFLEX_PAIRS
    )

    @classmethod
    def circumflex_regulator(cls, name: str) -> str:
        """Normalize circumflex letters in the name."""
        # 如果 name 不是字符串，将其转换为空字符串
        if not isinstance(name, str) or pd.isna(name):
            return ''

        # 执行字符串正则替换
        for regex_, format_character in cls.CIRCUMFLEX_REGEX_PAIRS:
            name = regex_.sub(format_character, name)
        return name


class CompanyMatcher:
    CJK_CHARACTERS = r'\u1100-\u11ff\u2e80-\u2fff\u3040-\u31ff\u3400-\u9fff\ua960-\ua97f\uac00-\ud7ff\uf900-\ufaff'
    CJK_OR_NUMERIC_REGEX = re.compile(
        rf"(?P<cjk>[{CJK_CHARACTERS}]+)|(?P<numeric>((?<=^\D)|(?<=[^\W0-9_]|\s))(?<!\b[a-zA-Z])(\d+([\W_]{{0,5}}\d+){{0,5}})(?=($|[^\W0-9_]|\s)))")
    WHITESPACE_REGEX = re.compile(r'\s+')
    COMPANY_COMMON_WORDS_REGEX = re.compile(
        r"(实验室|研究院|技术|科技)|(\b(?:laboratory|laboratories|lab|labs|research|technology|tech|science|sci)(\.|\b|$))",
        re.I
    )
    PREPOSITION_WORDS = {
        'aboard', 'about', 'above', 'across', 'after', 'against', 'along', 'amid', 'among', 'anti', 'around', 'as',
        'at',
        'before', 'behind', 'below', 'beneath', 'beside', 'besides', 'between', 'beyond', 'but', 'by',
        'concerning', 'considering', 'despite', 'down', 'during', 'except', 'excepting', 'excluding', 'following',
        'for', 'from', 'in', 'inside', 'into', 'like', 'minus', 'near', 'of', 'off', 'on', 'onto', 'opposite',
        'outside', 'over', 'past', 'per', 'plus', 'regarding', 'round', 'since', 'than', 'through', 'till', 'to',
        'toward', 'towards', 'under', 'underneath', 'unlike', 'until', 'up', 'upon', 'versus', 'via', 'with', 'within',
        'without'
    }
    CONJUNCTIONS_WORDS = {
        "although", "because", "before", "both", "and", '&', "whether", "or", "either", "neither", "nor", "once",
        "just", "so", "as", "if", "then", "rather", "than", "till", 'when', 'where', 'whenever', 'while', 'wherever',
        "such", "so", "that"
    }
    ARTICLES = {'a', 'an', 'the'}
    INLINE_WORDS = PREPOSITION_WORDS | CONJUNCTIONS_WORDS | ARTICLES

    ORG_SUFFIX_WORDS = {
        'zh': {'common': ('公司', '集团', '有限公司', '有限责任公司', '股份有限公司'),
               'uncommon': ('总公司', '股份', '控股', '责任', '有限', '企业', '协会', '合作社', '株式会社')},
        'en': {'common': ('company', 'group', 'corporation', 'incorporated', 'enterprise', 'enterprises',
                          'co', 'inc', 'corp', 'ltd', 'llc', 'se', 'pvt'),
               'uncommon': ('corporation limited', 'companies', 'worldwide', 'limited', "holding", "holdings",
                            'com', 'gmbh', 'ag', 'plc', 'sal', 'spa', r's\.p\.a', 'sab cv', 'sa', 'nv', r'n\.v', 'lp',
                            'sro',
                            'kg', 'aktiengesellschaft', 'de cv', 'ltda', "group of companies")}
    }
    ORG_SUFFIX_REGEX = re.compile(
        r"{0}|{1}".format(
            '|'.join([rf'{suf}' for suf in ORG_SUFFIX_WORDS['zh']['uncommon'] + ORG_SUFFIX_WORDS['zh']['common']]),
            rf"(?:\b(?:{'|'.join([suf for suf in ORG_SUFFIX_WORDS['en']['uncommon'] + ORG_SUFFIX_WORDS['en']['common']])})(?:\.|$))"
        ),
        re.IGNORECASE
    )

    COMPANY_KEYWORDS_REGEX = re.compile(
        r'\b(bank|college|university|education|energy|finance|army|air force|navy|industry|'
        r'hospital|hotel|institutes?|institution|petroleum|oil|health|electronic|commercial|'
        r'environmental|government|state|'
        r'global|market(?:ing)?|real estate|system)\b',
        re.IGNORECASE
    )

    def __init__(self, company_data_file: str):
        self.company_data_file = company_data_file
        self.matcher = NameMatcher(remove_ascii=False, punctuations=False)
        self.matching_data = None

    @staticmethod
    def contains_chinese(text):
        """Helper function to check if a string contains Chinese characters."""
        CHINESE_CHARACTERS = r'\u2e80-\u2fff\u31c0-\u31ef\u3400-\u9fff\uf900-\ufaff'
        return re.search(f"[{CHINESE_CHARACTERS}]", text)

    def company_words_regulator(self, company_name: str) -> str:
        """Regulate company names using multiple techniques."""
        # Step 1: Normalize circumflex letters
        search_str = NameNormalizer.circumflex_regulator(company_name)

        # Step 2: Remove spaces if only Chinese is detected
        if self.contains_chinese(search_str) and not re.search(r"[a-zA-Z]", search_str):
            search_str = search_str.replace(' ', '')

        # Step 3: Remove unnecessary company suffixes or common words
        search_str = self.WHITESPACE_REGEX.sub(' ', self.ORG_SUFFIX_REGEX.sub('', search_str)).strip()

        # Step 4: Filter out prepositions, conjunctions, and articles (INLINE_WORDS)
        words = search_str.split()
        filtered_words = [word for word in words if word.lower() not in self.INLINE_WORDS]
        search_str = ' '.join(filtered_words)

        # Step 5: Check for essential company keywords
        if self.COMPANY_KEYWORDS_REGEX.search(search_str):
            return search_str

        return search_str

    def load_company_data(self) -> pd.DataFrame:
        """Load the company data from a JSONL file."""
        companies = []
        with open(self.company_data_file, 'r', encoding='utf-8') as f:
            for line in f:
                companies.append(json.loads(line))
        return pd.DataFrame(companies)

    def prepare_matching_data(self, companies: pd.DataFrame) -> pd.DataFrame:
        """Prepare data for matching."""
        rows = []
        for _, company in companies.iterrows():
            # Process required search strings
            required_search_strings = company.get("requiredSearchStrings", [])
            if not isinstance(required_search_strings, list):
                required_search_strings = []  # Ensure it's a list if not iterable

            for search_string in required_search_strings:
                rows.append({"name": search_string, "companyName": company["companyName"]})

            # Process aliases
            aliases = company.get("aliases", [])
            if not isinstance(aliases, list):
                aliases = []  # Ensure aliases is a list if it's not iterable

            for alias in aliases:
                rows.append({"name": alias, "companyName": company["companyName"]})

        return pd.DataFrame(rows)

    def tokenize(self, text: str) -> str:
        """Tokenize the text using Jieba for Chinese words."""
        return " ".join(jieba.cut(text))

    def prepare_data_for_matching(self, companies: pd.DataFrame):
        """Tokenize and prepare company data for matching."""
        companies['name'] = companies['name'].apply(self.tokenize)
        self.matcher.load_and_process_master_data(column="name", df_matching_data=companies)

    def match_user_input(self, user_input: str) -> str:
        """Match user input against the company names."""

        # 进行匹配
        user_input_segmented = self.company_words_regulator(user_input)
        print(f"After Company_Words_Regulator user input: {user_input_segmented}")
        user_input_segmented = " ".join(jieba.cut(user_input_segmented))
        print(f"After Jieba user input: {user_input_segmented}")
        to_be_matched = pd.DataFrame({"name": [user_input_segmented]})
        result = self.matcher.match_names(to_be_matched, column_matching="name")

        # 如果没有匹配结果，则返回空字符串
        if result.empty:
            return ''

        # 获取第一个匹配结果
        best_match = result.iloc[0]["match_name"]
        match_score = result.iloc[0]["score"]

        # 如果匹配得分大于等于 60%，返回公司名称，否则返回空字符串
        if match_score >= 60:
            company_name = self.matching_data[self.matching_data['name'] == best_match]["companyName"].values[0]
            return company_name
        else:
            return ''


# Usage example:
def main():
    company_matcher = CompanyMatcher('bd_companies_international.jsonl')
    user_input = input("Enter a company name: ")
    matched_company = company_matcher.process(user_input)
    print(matched_company)


if __name__ == "__main__":
    main()


In [ ]:
#V9 Unittest
import unittest
import pandas as pd
from name_matching_for_company import CompanyMatcher
import csv
import time


class NameNormalizer:
    @classmethod
    def circumflex_regulator(cls, name: str) -> str:
        """Normalize circumflex letters in the name."""
        # 如果 name 不是字符串，将其转换为空字符串
        if not isinstance(name, str) or pd.isna(name):
            return ''

        # 执行字符串正则替换
        for regex_, format_character in cls.CIRCUMFLEX_REGEX_PAIRS:
            name = regex_.sub(format_character, name)
        return name


class TestCompanyMatcher(unittest.TestCase):

    @classmethod
    def setUpClass(cls):
        """Set up the CompanyMatcher and load the data once for all tests."""
        # 初始化 CompanyMatcher 并加载数据，只进行一次数据加载和 fit
        cls.matcher = CompanyMatcher('bd_companies_international.jsonl')  # 确保文件路径正确
        cls.matcher_data_loaded = False

        # 只加载一次数据，避免每次都重复加载
        cls.companies = cls.matcher.load_company_data()

        # 调用一次 prepare_matching_data，进行数据处理和匹配初始化
        cls.matcher.matching_data = cls.matcher.prepare_matching_data(cls.companies)  # 确保 matching_data 被赋值

        # 进行一次数据的处理和 fit 操作
        cls.matcher.prepare_data_for_matching(cls.matcher.matching_data)

        cls.matcher_data_loaded = True  # 确保所有数据已经准备好

    def test_company_name_matching(self):
        """Test the company name matching with input from the CSV file."""
        # 读取 filtered_company_names.csv 文件
        test_data = pd.read_csv('filtered_company_names.csv')

        # 准备输出结果 CSV 文件
        with open('company_name_matching_results.csv', 'w', newline='', encoding='utf-8') as output_file:
            writer = csv.writer(output_file)
            writer.writerow(['company name', 'expected company name', 'output', 'true/false', 'time (seconds)'])

            # 遍历每一行测试数据
            for index, row in test_data.iterrows():
                input_name = row['company name']
                expected_name = row['expected company name']

                # 记录开始时间
                start_time = time.time()

                # 调用 process 进行公司名匹配，避免每次都重复加载或处理数据
                output_name = self.matcher.match_user_input(input_name)

                # 记录结束时间
                end_time = time.time()

                # 计算处理时间
                elapsed_time = end_time - start_time

                # 确保 expected_name 和 output_name 为字符串，防止 AttributeError
                if pd.isna(expected_name):
                    expected_name = ''
                if pd.isna(output_name):
                    output_name = ''

                # 如果 output 和 expected 都是空，则设置 result 为 True
                if output_name == '' and expected_name == '':
                    result = True
                else:
                    result = output_name.lower() == expected_name.lower()

                # 写入结果到输出 CSV
                writer.writerow([input_name, expected_name, output_name, result, elapsed_time])


if __name__ == '__main__':
    unittest.main()


In [ ]:
#V10A
#Working code with aliases match
import re
import pandas as pd
import json
import jieba
from name_matching.name_matcher import NameMatcher


class NameNormalizer:
    CIRCUMFLEX_PAIRS = (
        (r'àâáãāåäǎ', 'a'), (r'ÀÂÁÃĀÅÄǍ', 'A'), (r'ß', 'b'), (r'čćç', 'c'), (r'ČÇ', 'C'),
        (r'ðđ', 'd'), (r'ÐĐ', 'D'), (r'éěèêëėệē', 'e'), (r'ÉÈÊËĖỆĒ', 'E'), (r'ğ', 'g'),
        (r'Ğ', 'G'), (r'îïíīịìǐĩı', 'i'), (r'İÎÏÍĪỊÌǏ', 'I'), (r'Ł', 'L'), (r'ł', 'l'),
        (r'ñňń', 'n'), (r'ÑŇŃ', 'N'), (r'ôöōồǒóòøőõ', 'o'), (r'ÔÖŌỒÓÒØŐÕ', 'O'), (r'ŕ', 'r'),
        (r'şš', 's'), (r'ŠŞ', 'S'), (r'ť', 't'), (r'ûùüúǔưū', 'u'), (r'ÛÙÜÚƯŪ', 'U'),
        (r'ŵ', 'w'), (r'ý', 'y'), (r'Ý', 'Y'), (r'žź', 'z'), (r'ŽŹ', 'Z'), (r'æ', 'ae'), (r'Æ', 'AE')
    )
    CIRCUMFLEX_REGEX_PAIRS = tuple(
        (re.compile(rf'[{regex_}]'), format_) for (regex_, format_) in CIRCUMFLEX_PAIRS
    )

    @classmethod
    def circumflex_regulator(cls, name: str) -> str:
        """Normalize circumflex letters in the name."""
        # 如果 name 不是字符串，将其转换为空字符串
        if not isinstance(name, str) or pd.isna(name):
            return ''

        # 执行字符串正则替换
        for regex_, format_character in cls.CIRCUMFLEX_REGEX_PAIRS:
            name = regex_.sub(format_character, name)
        return name


class CompanyMatcher:
    CJK_CHARACTERS = r'\u1100-\u11ff\u2e80-\u2fff\u3040-\u31ff\u3400-\u9fff\ua960-\ua97f\uac00-\ud7ff\uf900-\ufaff'
    CJK_OR_NUMERIC_REGEX = re.compile(
        rf"(?P<cjk>[{CJK_CHARACTERS}]+)|(?P<numeric>((?<=^\D)|(?<=[^\W0-9_]|\s))(?<!\b[a-zA-Z])(\d+([\W_]{{0,5}}\d+){{0,5}})(?=($|[^\W0-9_]|\s)))")
    WHITESPACE_REGEX = re.compile(r'\s+')
    COMPANY_COMMON_WORDS_REGEX = re.compile(
        r"(实验室|研究院|技术|科技)|(\b(?:laboratory|laboratories|lab|labs|research|technology|tech|science|sci)(\.|\b|$))",
        re.I
    )
    PREPOSITION_WORDS = {
        'aboard', 'about', 'above', 'across', 'after', 'against', 'along', 'amid', 'among', 'anti', 'around', 'as',
        'at',
        'before', 'behind', 'below', 'beneath', 'beside', 'besides', 'between', 'beyond', 'but', 'by',
        'concerning', 'considering', 'despite', 'down', 'during', 'except', 'excepting', 'excluding', 'following',
        'for', 'from', 'in', 'inside', 'into', 'like', 'minus', 'near', 'of', 'off', 'on', 'onto', 'opposite',
        'outside', 'over', 'past', 'per', 'plus', 'regarding', 'round', 'since', 'than', 'through', 'till', 'to',
        'toward', 'towards', 'under', 'underneath', 'unlike', 'until', 'up', 'upon', 'versus', 'via', 'with', 'within',
        'without'
    }
    CONJUNCTIONS_WORDS = {
        "although", "because", "before", "both", "and", '&', "whether", "or", "either", "neither", "nor", "once",
        "just", "so", "as", "if", "then", "rather", "than", "till", 'when', 'where', 'whenever', 'while', 'wherever',
        "such", "so", "that"
    }
    ARTICLES = {'a', 'an', 'the'}
    INLINE_WORDS = PREPOSITION_WORDS | CONJUNCTIONS_WORDS | ARTICLES

    ORG_SUFFIX_WORDS = {
        'zh': {'common': ('公司', '集团', '有限公司', '有限责任公司', '股份有限公司'),
               'uncommon': ('总公司', '股份', '控股', '责任', '有限', '企业', '协会', '合作社', '株式会社')},
        'en': {'common': ('company', 'group', 'corporation', 'incorporated', 'enterprise', 'enterprises',
                          'co', 'inc', 'corp', 'ltd', 'llc', 'se', 'pvt'),
               'uncommon': ('corporation limited', 'companies', 'worldwide', 'limited', "holding", "holdings",
                            'com', 'gmbh', 'ag', 'plc', 'sal', 'spa', r's\.p\.a', 'sab cv', 'sa', 'nv', r'n\.v', 'lp',
                            'sro',
                            'kg', 'aktiengesellschaft', 'de cv', 'ltda', "group of companies")}
    }
    ORG_SUFFIX_REGEX = re.compile(
        r"{0}|{1}".format(
            '|'.join([rf'{suf}' for suf in ORG_SUFFIX_WORDS['zh']['uncommon'] + ORG_SUFFIX_WORDS['zh']['common']]),
            rf"(?:\b(?:{'|'.join([suf for suf in ORG_SUFFIX_WORDS['en']['uncommon'] + ORG_SUFFIX_WORDS['en']['common']])})(?:\.|$))"
        ),
        re.IGNORECASE
    )

    COMPANY_KEYWORDS_REGEX = re.compile(
        r'\b(bank|college|university|education|energy|finance|army|air force|navy|industry|'
        r'hospital|hotel|institutes?|institution|petroleum|oil|health|electronic|commercial|'
        r'environmental|government|state|'
        r'global|market(?:ing)?|real estate|system)\b',
        re.IGNORECASE
    )

    def __init__(self, company_data_file: str):
        self.company_data_file = company_data_file
        self.matcher = NameMatcher(remove_ascii=False, punctuations=False)
        self.matching_data = None

    @staticmethod
    def contains_chinese(text):
        """Helper function to check if a string contains Chinese characters."""
        CHINESE_CHARACTERS = r'\u2e80-\u2fff\u31c0-\u31ef\u3400-\u9fff\uf900-\ufaff'
        return re.search(f"[{CHINESE_CHARACTERS}]", text)

    def company_words_regulator(self, company_name: str) -> str:
        """Regulate company names using multiple techniques."""
        # Step 1: Normalize circumflex letters
        search_str = NameNormalizer.circumflex_regulator(company_name)

        # Step 2: Remove spaces if only Chinese is detected
        if self.contains_chinese(search_str) and not re.search(r"[a-zA-Z]", search_str):
            search_str = search_str.replace(' ', '')

        # Step 3: Remove unnecessary company suffixes or common words
        search_str = self.WHITESPACE_REGEX.sub(' ', self.ORG_SUFFIX_REGEX.sub('', search_str)).strip()

        # Step 4: Filter out prepositions, conjunctions, and articles (INLINE_WORDS)
        words = search_str.split()
        filtered_words = [word for word in words if word.lower() not in self.INLINE_WORDS]
        search_str = ' '.join(filtered_words)

        # Step 5: Check for essential company keywords
        if self.COMPANY_KEYWORDS_REGEX.search(search_str):
            return search_str

        return search_str

    def load_company_data(self) -> pd.DataFrame:
        """加载公司数据并确保别名信息正确存在."""
        companies = []
        with open(self.company_data_file, 'r', encoding='utf-8') as f:
            for line in f:
                company = json.loads(line)
                company['aliases'] = company.get('aliases', [])  # 确保别名存在
                companies.append(company)
        return pd.DataFrame(companies)

    def prepare_matching_data(self, companies: pd.DataFrame) -> pd.DataFrame:
        """Prepare data for matching, ensuring aliases and required search strings are correctly loaded as individual entries."""
        rows = []
        for _, company in companies.iterrows():
            # Process required search strings
            required_search_strings = company.get("requiredSearchStrings", [])
            if not isinstance(required_search_strings, list):
                required_search_strings = []  # Ensure it's a list if not iterable

            # Combine required search strings and aliases into a full alias list
            aliases = company.get("aliases", [])
            if not isinstance(aliases, list):
                aliases = []  # Ensure aliases is a list if it's not iterable

            # 将所有 requiredSearchStrings 和 aliases 合并为一个别名列表
            full_alias_list = required_search_strings + aliases

            print(f"Processing full aliases list for company {company['companyName']}: {full_alias_list}")

            # For each alias, create a separate row with all aliases as alias_name
            for alias in full_alias_list:
                rows.append({
                    "name": alias,
                    "companyName": company.get("companyName", ""),
                    "alias_name": full_alias_list,  # 将完整的别名列表作为 alias_name 存储
                    "is_alias": alias in aliases  # 标记是否为别名
                })

        matching_data = pd.DataFrame(rows)

        # 打印生成的匹配数据样本，确认别名被正确加载
        print("Sample of matching data after processing full aliases:")
        print(matching_data[['name', 'companyName', 'alias_name']].head(10))

        return matching_data

    def tokenize(self, text: str) -> str:
        """Tokenize the text using Jieba for Chinese words."""
        return " ".join(jieba.cut(text))

    def prepare_data_for_matching(self, companies: pd.DataFrame):
        """Tokenize and prepare company data for matching."""
        # 对公司数据进行分词处理
        companies['name'] = companies['name'].apply(self.tokenize)

        # Load and process master data for matching
        self.matcher.load_and_process_master_data(column="name", df_matching_data=companies)  # 确保主数据加载

    def match_user_input(self, user_input: str, expected_name: str) -> str:
        """Match user input against the company names, then validate against expected name."""

        # 对输入进行标准化和分词处理
        print(f"User input: {user_input}")  # 打印调试信息
        user_input_segmented = self.company_words_regulator(user_input)
        print(f"Segmented input: {user_input_segmented}")  # 打印调试信息
        user_input_segmented = " ".join(jieba.cut(user_input_segmented))
        print(f"Segmented and tokenized input: {user_input_segmented}")  # 打印调试信息
        to_be_matched = pd.DataFrame({"name": [user_input_segmented]})

        # 匹配用户输入的名称
        result = self.matcher.match_names(to_be_matched, column_matching="name")
        print(f"Matching result: {result}")  # 打印调试信息

        # 如果没有匹配结果，返回空字符串
        if result.empty:
            print("No match found")  # 打印调试信息
            return ''

        # 获取最匹配的结果
        best_match = result.iloc[0]["match_name"]
        match_score = result.iloc[0]["score"]
        print(f"Best match: {best_match}, Match score: {match_score}")  # 打印调试信息

        # 匹配得分超过 60%，视为成功匹配
        if match_score >= 70:
            # 通过 best_match 查找公司名称和别名
            matched_row = self.matching_data[self.matching_data['name'] == best_match]
            if matched_row.empty:
                print(f"No matching row found for: {best_match}")  # 打印调试信息
                return ''

            # 获取公司名称和别名信息
            company_name = matched_row.iloc[0]['companyName']
            aliases = matched_row.iloc[0]['alias_name']  # 这现在是一个包含所有别名的列表

            # 打印公司名称和别名
            print(f"Company name: {company_name}, Aliases: {aliases}")  # 打印当前公司名称和别名

            # 处理 expected_name，防止 NaN 导致错误
            if pd.isna(expected_name):
                expected_name = ""  # 如果是 NaN，替换为空字符串
            else:
                expected_name = expected_name.lower()

            print(f"Expected name: {expected_name}")  # 打印 expected_name

            # 检查 expected_name 是否在别名中
            if expected_name and expected_name in [alias.lower() for alias in aliases]:
                print(f"Matched alias: {expected_name}")  # 打印匹配到的别名
                return expected_name

            # 如果 expected_name 和 company_name 匹配，返回公司名称
            if expected_name == company_name.lower():
                print(f"Returning company name: {company_name}")  # 打印调试信息
                return company_name

            # 如果没有匹配到 expected_name，默认返回公司名称
            print(f"Returning default company name: {company_name}")  # 打印调试信息
            return company_name
        else:
            print("Match score below threshold, returning empty")  # 打印调试信息
            return ''


# Usage example:
def main():
    # 初始化 CompanyMatcher
    company_matcher = CompanyMatcher('bd_companies_international.jsonl')

    # 加载公司数据
    companies = company_matcher.load_company_data()

    # 准备匹配数据，并且加载到 NameMatcher 中
    matching_data = company_matcher.prepare_matching_data(companies)
    company_matcher.matching_data = matching_data  # 确保 matching_data 被正确赋值

    company_matcher.prepare_data_for_matching(matching_data)  # 加载并处理主数据

    # 获取用户输入并进行匹配
    user_input = input("Enter a company name: ")
    
    # 使用 match_user_input 来进行公司匹配
    matched_company = company_matcher.match_user_input(user_input, expected_name=None)  # 如果没有预期值，可以传入 None
    print(matched_company)



if __name__ == "__main__":
    main()

In [ ]:
#V10A Unittest
#Working Unittest code with aliases match
import unittest
import pandas as pd
from name_matching_for_company import CompanyMatcher
import csv
import time


class NameNormalizer:
    @classmethod
    def circumflex_regulator(cls, name: str) -> str:
        """Normalize circumflex letters in the name."""
        # 如果 name 不是字符串，将其转换为空字符串
        if not isinstance(name, str) or pd.isna(name):
            return ''

        # 执行字符串正则替换
        for regex_, format_character in cls.CIRCUMFLEX_REGEX_PAIRS:
            name = regex_.sub(format_character, name)
        return name


class TestCompanyMatcher(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        """Set up the CompanyMatcher and load the data once for all tests."""
        cls.matcher = CompanyMatcher('bd_companies_international.jsonl')

        # 加载公司数据
        cls.companies = cls.matcher.load_company_data()

        # Prepare matching data
        cls.matcher.matching_data = cls.matcher.prepare_matching_data(cls.companies)

        # Load and process the master data for matching
        cls.matcher.prepare_data_for_matching(cls.matcher.matching_data)  # 加载并处理主数据

        # 打印 matching_data 样本，检查别名是否加载正确
        print("Sample of matching data in setUpClass:")
        print(cls.matcher.matching_data[['name', 'companyName', 'alias_name']].head(10))

    def test_company_name_matching(self):
        """Test the company name matching with input from the CSV file."""
        test_data = pd.read_csv('filtered_company_names.csv')

        with open('company_name_matching_results.csv', 'w', newline='', encoding='utf-8') as output_file:
            writer = csv.writer(output_file)
            writer.writerow(['company name', 'expected company name', 'output', 'true/false', 'time (seconds)'])

            for index, row in test_data.iterrows():
                input_name = row['company name']
                expected_name = row['expected company name']

                start_time = time.time()

                # Match the user input with the company name or alias
                output_name = self.matcher.match_user_input(input_name, expected_name)  # 传入 expected_name 参数

                end_time = time.time()
                elapsed_time = end_time - start_time

                # 确保 expected_name 和 output_name 为字符串，防止 AttributeError
                if pd.isna(expected_name):
                    expected_name = ''
                if pd.isna(output_name):
                    output_name = ''

                # 如果 output 和 expected 都是空，则设置 result 为 True
                if output_name == '' and expected_name == '':
                    result = True
                else:
                    result = output_name.lower() == expected_name.lower()

                # 写入结果到输出 CSV
                writer.writerow([input_name, expected_name, output_name, result, elapsed_time])


if __name__ == '__main__':
    unittest.main()


In [ ]:
#V10B
#match_score changes to threshold, threshold will be difined in V10B unittest
import re
import pandas as pd
import json
import jieba
from name_matching.name_matcher import NameMatcher


class NameNormalizer:
    """
    CompanyMatcher is responsible for matching company names against a list of known companies and their aliases.

    Attributes:
    - company_data_file (str): Path to the JSONL file containing company data.
    - matcher (NameMatcher): A NameMatcher object used for name matching.
    - matching_data (pd.DataFrame): DataFrame storing processed company names and aliases.
    """
    CIRCUMFLEX_PAIRS = (
        (r'àâáãāåäǎ', 'a'), (r'ÀÂÁÃĀÅÄǍ', 'A'), (r'ß', 'b'), (r'čćç', 'c'), (r'ČÇ', 'C'),
        (r'ðđ', 'd'), (r'ÐĐ', 'D'), (r'éěèêëėệē', 'e'), (r'ÉÈÊËĖỆĒ', 'E'), (r'ğ', 'g'),
        (r'Ğ', 'G'), (r'îïíīịìǐĩı', 'i'), (r'İÎÏÍĪỊÌǏ', 'I'), (r'Ł', 'L'), (r'ł', 'l'),
        (r'ñňń', 'n'), (r'ÑŇŃ', 'N'), (r'ôöōồǒóòøőõ', 'o'), (r'ÔÖŌỒÓÒØŐÕ', 'O'), (r'ŕ', 'r'),
        (r'şš', 's'), (r'ŠŞ', 'S'), (r'ť', 't'), (r'ûùüúǔưū', 'u'), (r'ÛÙÜÚƯŪ', 'U'),
        (r'ŵ', 'w'), (r'ý', 'y'), (r'Ý', 'Y'), (r'žź', 'z'), (r'ŽŹ', 'Z'), (r'æ', 'ae'), (r'Æ', 'AE')
    )
    CIRCUMFLEX_REGEX_PAIRS = tuple(
        (re.compile(rf'[{regex_}]'), format_) for (regex_, format_) in CIRCUMFLEX_PAIRS
    )

    @classmethod
    def circumflex_regulator(cls, name: str) -> str:
        """Normalize circumflex letters in the name."""
        # 如果 name 不是字符串，将其转换为空字符串
        if not isinstance(name, str) or pd.isna(name):
            return ''

        # 执行字符串正则替换
        for regex_, format_character in cls.CIRCUMFLEX_REGEX_PAIRS:
            name = regex_.sub(format_character, name)
        return name


class CompanyMatcher:
    CJK_CHARACTERS = r'\u1100-\u11ff\u2e80-\u2fff\u3040-\u31ff\u3400-\u9fff\ua960-\ua97f\uac00-\ud7ff\uf900-\ufaff'
    CJK_OR_NUMERIC_REGEX = re.compile(
        rf"(?P<cjk>[{CJK_CHARACTERS}]+)|(?P<numeric>((?<=^\D)|(?<=[^\W0-9_]|\s))(?<!\b[a-zA-Z])(\d+([\W_]{{0,5}}\d+){{0,5}})(?=($|[^\W0-9_]|\s)))")
    WHITESPACE_REGEX = re.compile(r'\s+')
    COMPANY_COMMON_WORDS_REGEX = re.compile(
        r"(实验室|研究院|技术|科技)|(\b(?:laboratory|laboratories|lab|labs|research|technology|tech|science|sci)(\.|\b|$))",
        re.I
    )
    PREPOSITION_WORDS = {
        'aboard', 'about', 'above', 'across', 'after', 'against', 'along', 'amid', 'among', 'anti', 'around', 'as',
        'at',
        'before', 'behind', 'below', 'beneath', 'beside', 'besides', 'between', 'beyond', 'but', 'by',
        'concerning', 'considering', 'despite', 'down', 'during', 'except', 'excepting', 'excluding', 'following',
        'for', 'from', 'in', 'inside', 'into', 'like', 'minus', 'near', 'of', 'off', 'on', 'onto', 'opposite',
        'outside', 'over', 'past', 'per', 'plus', 'regarding', 'round', 'since', 'than', 'through', 'till', 'to',
        'toward', 'towards', 'under', 'underneath', 'unlike', 'until', 'up', 'upon', 'versus', 'via', 'with', 'within',
        'without'
    }
    CONJUNCTIONS_WORDS = {
        "although", "because", "before", "both", "and", '&', "whether", "or", "either", "neither", "nor", "once",
        "just", "so", "as", "if", "then", "rather", "than", "till", 'when', 'where', 'whenever', 'while', 'wherever',
        "such", "so", "that"
    }
    ARTICLES = {'a', 'an', 'the'}
    INLINE_WORDS = PREPOSITION_WORDS | CONJUNCTIONS_WORDS | ARTICLES

    ORG_SUFFIX_WORDS = {
        'zh': {'common': ('公司', '集团', '有限公司', '有限责任公司', '股份有限公司'),
               'uncommon': ('总公司', '股份', '控股', '责任', '有限', '企业', '协会', '合作社', '株式会社')},
        'en': {'common': ('company', 'group', 'corporation', 'incorporated', 'enterprise', 'enterprises',
                          'co', 'inc', 'corp', 'ltd', 'llc', 'se', 'pvt'),
               'uncommon': ('corporation limited', 'companies', 'worldwide', 'limited', "holding", "holdings",
                            'com', 'gmbh', 'ag', 'plc', 'sal', 'spa', r's\.p\.a', 'sab cv', 'sa', 'nv', r'n\.v', 'lp',
                            'sro',
                            'kg', 'aktiengesellschaft', 'de cv', 'ltda', "group of companies")}
    }
    ORG_SUFFIX_REGEX = re.compile(
        r"{0}|{1}".format(
            '|'.join([rf'{suf}' for suf in ORG_SUFFIX_WORDS['zh']['uncommon'] + ORG_SUFFIX_WORDS['zh']['common']]),
            rf"(?:\b(?:{'|'.join([suf for suf in ORG_SUFFIX_WORDS['en']['uncommon'] + ORG_SUFFIX_WORDS['en']['common']])})(?:\.|$))"
        ),
        re.IGNORECASE
    )

    COMPANY_KEYWORDS_REGEX = re.compile(
        r'\b(bank|college|university|education|energy|finance|army|air force|navy|industry|'
        r'hospital|hotel|institutes?|institution|petroleum|oil|health|electronic|commercial|'
        r'environmental|government|state|'
        r'global|market(?:ing)?|real estate|system)\b',
        re.IGNORECASE
    )

    def __init__(self, company_data_file: str):
        self.company_data_file = company_data_file
        self.matcher = NameMatcher(remove_ascii=False, punctuations=False)
        self.matching_data = None

    @staticmethod
    def contains_chinese(text):
        """Helper function to check if a string contains Chinese characters."""
        CHINESE_CHARACTERS = r'\u2e80-\u2fff\u31c0-\u31ef\u3400-\u9fff\uf900-\ufaff'
        return re.search(f"[{CHINESE_CHARACTERS}]", text)

    def company_words_regulator(self, company_name: str) -> str:
        """Regulate company names using multiple techniques."""
        # Step 1: Normalize circumflex letters
        search_str = NameNormalizer.circumflex_regulator(company_name)

        # Step 2: Remove spaces if only Chinese is detected
        if self.contains_chinese(search_str) and not re.search(r"[a-zA-Z]", search_str):
            search_str = search_str.replace(' ', '')

        # Step 3: Remove unnecessary company suffixes or common words
        search_str = self.WHITESPACE_REGEX.sub(' ', self.ORG_SUFFIX_REGEX.sub('', search_str)).strip()

        # Step 4: Filter out prepositions, conjunctions, and articles (INLINE_WORDS)
        words = search_str.split()
        filtered_words = [word for word in words if word.lower() not in self.INLINE_WORDS]
        search_str = ' '.join(filtered_words)

        # Step 5: Check for essential company keywords
        if self.COMPANY_KEYWORDS_REGEX.search(search_str):
            return search_str

        return search_str

    def load_company_data(self) -> pd.DataFrame:
        """加载公司数据并确保别名信息正确存在."""
        companies = []
        with open(self.company_data_file, 'r', encoding='utf-8') as f:
            for line in f:
                company = json.loads(line)
                company['aliases'] = company.get('aliases', [])  # 确保别名存在
                companies.append(company)
        return pd.DataFrame(companies)

    def prepare_matching_data(self, companies: pd.DataFrame) -> pd.DataFrame:
        """Prepare data for matching, ensuring aliases and required search strings are correctly loaded as individual entries."""
        rows = []
        for _, company in companies.iterrows():
            # Process required search strings
            required_search_strings = company.get("requiredSearchStrings", [])
            if not isinstance(required_search_strings, list):
                required_search_strings = []  # Ensure it's a list if not iterable

            # Combine required search strings and aliases into a full alias list
            aliases = company.get("aliases", [])
            if not isinstance(aliases, list):
                aliases = []  # Ensure aliases is a list if it's not iterable

            # Merge requiredSearchStrings and aliases into a full alias list
            full_alias_list = required_search_strings + aliases

            # For each alias, create a separate row with all aliases as alias_name
            for alias in full_alias_list:
                rows.append({
                    "name": alias,
                    "companyName": company.get("companyName", ""),
                    "alias_name": full_alias_list,  # 将完整的别名列表作为 alias_name 存储
                    "is_alias": alias in aliases  # 标记是否为别名
                })

        # Return the prepared matching data as a DataFrame
        matching_data = pd.DataFrame(rows)
        return matching_data

    def tokenize(self, text: str) -> str:
        """Tokenize the text using Jieba for Chinese words."""
        return " ".join(jieba.cut(text))

    def prepare_data_for_matching(self, companies: pd.DataFrame):
        """Tokenize and prepare company data for matching."""
        # 对公司数据进行分词处理
        companies['name'] = companies['name'].apply(self.tokenize)

        # Load and process master data for matching
        self.matcher.load_and_process_master_data(column="name", df_matching_data=companies)  # 确保主数据加载

    def match_user_input(self, user_input: str, expected_name: str, threshold: int = 70) -> str:
        """Match user input against the company names, then validate against expected name, with dynamic threshold."""

        # 对输入进行标准化和分词处理
        print(f"User input: {user_input}")  # 打印调试信息
        user_input_segmented = self.company_words_regulator(user_input)
        print(f"Segmented input: {user_input_segmented}")  # 打印调试信息
        user_input_segmented = " ".join(jieba.cut(user_input_segmented))
        print(f"Segmented and tokenized input: {user_input_segmented}")  # 打印调试信息
        to_be_matched = pd.DataFrame({"name": [user_input_segmented]})

        # 匹配用户输入的名称
        result = self.matcher.match_names(to_be_matched, column_matching="name")
        print(f"Matching result: {result}")  # 打印调试信息

        # 如果没有匹配结果，返回空字符串
        if result.empty:
            print("No match found")  # 打印调试信息
            return ''

        # 获取最匹配的结果
        best_match = result.iloc[0]["match_name"]
        match_score = result.iloc[0]["score"]
        print(f"Best match: {best_match}, Match score: {match_score}")  # 打印调试信息

        # 使用传入的动态 threshold 参数,match_score
        if match_score >= threshold:
            # 通过 best_match 查找公司名称和别名
            matched_row = self.matching_data[self.matching_data['name'] == best_match]
            if matched_row.empty:
                print(f"No matching row found for: {best_match}")  # 打印调试信息
                return ''

            # 获取公司名称和别名信息
            company_name = matched_row.iloc[0]['companyName']
            aliases = matched_row.iloc[0]['alias_name']  # 这现在是一个包含所有别名的列表

            # 打印公司名称和别名
            print(f"Company name: {company_name}, Aliases: {aliases}")  # 打印当前公司名称和别名

            # 处理 expected_name，防止 NaN 导致错误
            if pd.isna(expected_name):
                expected_name = ""  # 如果是 NaN，替换为空字符串
            else:
                expected_name = expected_name.lower()

            print(f"Expected name: {expected_name}")  # 打印 expected_name

            # 检查 expected_name 是否在别名中
            if expected_name and expected_name in [alias.lower() for alias in aliases]:
                print(f"Matched alias: {expected_name}")  # 打印匹配到的别名
                return expected_name

            # 如果 expected_name 和 company_name 匹配，返回公司名称
            if expected_name == company_name.lower():
                print(f"Returning company name: {company_name}")  # 打印调试信息
                return company_name

            # 如果没有匹配到 expected_name，默认返回公司名称
            print(f"Returning default company name: {company_name}")  # 打印调试信息
            return company_name
        else:
            print("Match score below threshold, returning empty")  # 打印调试信息
            return ''


# Usage example:
def main():
    # 初始化 CompanyMatcher
    company_matcher = CompanyMatcher('bd_companies_international.jsonl')

    # 加载公司数据
    companies = company_matcher.load_company_data()

    # 准备匹配数据，并且加载到 NameMatcher 中
    matching_data = company_matcher.prepare_matching_data(companies)
    company_matcher.matching_data = matching_data  # 确保 matching_data 被正确赋值

    company_matcher.prepare_data_for_matching(matching_data)  # 加载并处理主数据

    # 获取用户输入并进行匹配
    user_input = input("Enter a company name: ")

    # 使用 match_user_input 来进行公司匹配
    matched_company = company_matcher.match_user_input(user_input, expected_name=None)  # 如果没有预期值，可以传入 None
    print(matched_company)


if __name__ == "__main__":
    main()


In [ ]:
#V10B unittest
#This is for people who want's to test multiple match_score instead of just one
import unittest
import pandas as pd
import csv
import time
from name_matching_for_company import CompanyMatcher
class TestCompanyMatcher(unittest.TestCase):

    @classmethod
    def setUpClass(cls):
        """Set up the CompanyMatcher and load the data once for all tests."""
        cls.matcher = CompanyMatcher('bd_companies_international.jsonl')
        cls.companies = cls.matcher.load_company_data()
        cls.matcher.matching_data = cls.matcher.prepare_matching_data(cls.companies)
        cls.matcher.prepare_data_for_matching(cls.matcher.matching_data)

    def test_company_name_matching_dynamic_threshold(self):
        """Test the company name matching with dynamic threshold."""
        test_data = pd.read_csv('filtered_company_names.csv')

        # Loop from match_score >= 0 to match_score >= 100
        for threshold in range(0, 101):#This range can change to whatever you want
            with self.subTest(threshold=threshold):  # Use unittest subTest for each threshold
                # Output CSV file for this specific threshold
                output_filename = f'company_name_matching_results_threshold_{threshold}.csv'

                # Prepare output file
                with open(output_filename, 'w', newline='', encoding='utf-8') as output_file:
                    writer = csv.writer(output_file)
                    writer.writerow(['company name', 'expected company name', 'output', 'true/false', 'time (seconds)'])

                    # Iterate over test data
                    for index, row in test_data.iterrows():
                        input_name = row['company name']
                        expected_name = row['expected company name']

                        start_time = time.time()

                        # Call match_user_input with the dynamic threshold
                        output_name = self.matcher.match_user_input(input_name, expected_name, threshold=threshold)

                        end_time = time.time()
                        elapsed_time = end_time - start_time

                        # Ensure expected_name and output_name are strings
                        expected_name = '' if pd.isna(expected_name) else expected_name
                        output_name = '' if pd.isna(output_name) else output_name

                        # Determine if the output matches the expected name
                        result = output_name.lower() == expected_name.lower()

                        # Write the result to the CSV file
                        writer.writerow([input_name, expected_name, output_name, result, elapsed_time])

if __name__ == '__main__':
    unittest.main()



In [ ]:
#V10B Score and Graph calculation
#Calculate accuracy, precision, recall, and F1-score, also gives graph of these score
#This needs to run after V10B unittest
import pandas as pd
import matplotlib.pyplot as plt

# 定义处理的起始和结束阈值（可以灵活调整）
start_threshold = 0  # 起始阈值
end_threshold = 3  # 结束阈值

# 初始化空的列表，用来存储每个文件的结果
results = []

# 循环处理指定范围内的 CSV 文件
for i in range(start_threshold, end_threshold + 1):  # 范围是从 start 到 end
    # 构造文件名
    file_name = f'company_name_matching_results_threshold_{i}.csv'

    # 加载文件
    try:
        data = pd.read_csv(file_name)
    except FileNotFoundError:
        print(f"File {file_name} not found. Skipping...")
        continue

    # 计算 TP, FP, FN, TN
    tp = data[(data['true/false'] == True) & (data['output'].notna())].shape[0]
    fp = data[(data['true/false'] == False) & (data['output'].notna())].shape[0]
    fn = data[(data['true/false'] == False) & (data['output'].isna())].shape[0]
    tn = data[(data['true/false'] == True) & (data['output'].isna())].shape[0]

    # 计算 Precision, Accuracy, Recall, F1
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    accuracy = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # 将结果保存到字典中
    results.append({
        'threshold': i,
        'TP': tp,
        'FP': fp,
        'FN': fn,
        'TN': tn,
        'Precision': precision,
        'Accuracy': accuracy,
        'Recall': recall,
        'F1-score': f1_score
    })

# 将结果转换为 DataFrame 并保存到文件
results_df = pd.DataFrame(results)
results_df.to_csv('calculate_score.csv', index=False)

# 绘制 Precision, Recall, F1-score 和 Accuracy 曲线
plt.figure(figsize=(10, 6))
plt.plot(results_df['threshold'], results_df['Precision'], label='Precision', marker='o')
plt.plot(results_df['threshold'], results_df['Recall'], label='Recall', marker='o')
plt.plot(results_df['threshold'], results_df['F1-score'], label='F1-score', marker='o')
plt.plot(results_df['threshold'], results_df['Accuracy'], label='Accuracy', marker='o')

# 设置x轴为每隔5个显示一个刻度
plt.xticks(ticks=range(0, results_df['threshold'].max() + 1, 2))

# 设置标题和标签
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Performance Metrics vs Threshold')
plt.legend()
plt.grid(True)

# 显示图表
plt.show()


In [ ]:
#V11A
#Can Use V10A Unittest
#Add suffix process
import re
import pandas as pd
import json
import jieba
from name_matching.name_matcher import NameMatcher

# A regular expression to match and replace certain symbols with spaces, including slashes, percentage signs, exclamations, punctuation, hyphens, and special symbols.
ERASE_OR_REPLACE_TO_SPACE_SYMBOLS = re.compile(r'''[/\\%!！.?？^*·•⦁\-]''')

# A regular expression to match entity names that start with "the" followed by alphanumeric characters, case-insensitive.
THE_SPECIAL_ENTITY_NAME_REGEX = re.compile(r'^(the\s+[a-z0-9]+)$', re.IGNORECASE)

# A regular expression to match sequences of single letters separated by spaces (e.g., "a b c") and find those patterns, case-insensitive.
SINGLE_LETTER_SEQUENTIAL_REGEX = re.compile(rf'(\b[a-z]\b)(\s+[a-z]\b)+', re.IGNORECASE)

# A regular expression to match single lowercase letters surrounded by spaces (e.g., " a ").
SINGLE_LOWERCASE_REGEX = re.compile(rf'\s+[a-z]\s+')

# A regular expression to match and remove invalid symbols in company names, such as punctuation, special characters, and spaces around hyphens.
REGEX_COMPANY_DROP_INVALID = re.compile(r'[~$()@—－（）√《》:;：；.、•·‧・，*?+&|,/\[\\]|{}\t\n"]|([-]+\s|\s[-]+)')

# A regular expression to match symbols that should be replaced with spaces, including ampersands, question marks, asterisks, parentheses, commas, and hyphens.
REPLACE_TO_SPACE_SYMBOLS = re.compile(r'[&?#*|=_+()（）、,，–]')

class NameNormalizer:
    """
    This class handles the normalization of names, including circumflex letters and symbols.
    """

    # Pairing circumflex letters with their normalized counterparts
    CIRCUMFLEX_PAIRS = (
        (r'àâáãāåäǎ', 'a'), (r'ÀÂÁÃĀÅÄǍ', 'A'), (r'ß', 'b'), (r'čćç', 'c'), (r'ČÇ', 'C'),
        (r'ðđ', 'd'), (r'ÐĐ', 'D'), (r'éěèêëėệē', 'e'), (r'ÉÈÊËĖỆĒ', 'E'), (r'ğ', 'g'),
        (r'Ğ', 'G'), (r'îïíīịìǐĩı', 'i'), (r'İÎÏÍĪỊÌǏ', 'I'), (r'Ł', 'L'), (r'ł', 'l'),
        (r'ñňń', 'n'), (r'ÑŇŃ', 'N'), (r'ôöōồǒóòøőõ', 'o'), (r'ÔÖŌỒÓÒØŐÕ', 'O'), (r'ŕ', 'r'),
        (r'şš', 's'), (r'ŠŞ', 'S'), (r'ť', 't'), (r'ûùüúǔưū', 'u'), (r'ÛÙÜÚƯŪ', 'U'),
        (r'ŵ', 'w'), (r'ý', 'y'), (r'Ý', 'Y'), (r'žź', 'z'), (r'ŽŹ', 'Z'), (r'æ', 'ae'), (r'Æ', 'AE')
    )

    # Compile regex for circumflex replacements
    CIRCUMFLEX_REGEX_PAIRS = tuple(
        (re.compile(rf'[{regex_}]'), format_) for (regex_, format_) in CIRCUMFLEX_PAIRS
    )

    @classmethod
    def circumflex_regulator(cls, name: str) -> str:
        """
        Normalize circumflex letters in the name, e.g., 'é' becomes 'e'.
        If name is not a string, return an empty string.
        """
        if not isinstance(name, str) or pd.isna(name):
            return ''

        # Replace circumflex letters using pre-compiled regex pairs
        for regex_, format_character in cls.CIRCUMFLEX_REGEX_PAIRS:
            name = regex_.sub(format_character, name)
        return name


class CompanyMatcher:
    """
    This class handles company name matching using various techniques like tokenization, symbol regulation,
    and name normalization.
    """
    # Define common words for both English and Chinese
    COMPANY_COMMON_WORDS = {
        "en": {
            "laboratory", "laboratories", "lab", "labs", "research", "technology", "technologies",
            "technical", "tech", "sci", "science"
        },
        "zh": {
            "实验室", "研究院", "技术", "科技"
        }
    }
    COMPANY_KEYWORDS_REGEX = re.compile(
        r'\b(bank|college|university|education|energy|finance|army|air foce|navy|industry|'
        r'hospital|hotel|institutes?|institution|petroleum|oil|health|electronic|commercial|'
        r'environmental|gover(?:nor?|ment)|state|estado|procter|reliance|genome|software|hardware|agency|'
        r'(?:sou|nor)th(?:[-\s]?(?:ea|we)st(?:ern)?)?|[东西南北]方|[东西][南北][方]?|环境|'
        r'银行|软件|硬件|大学|机构|研究院|学院|教育|协会|能源|金融|旅馆|医院|石油|电子|工业|[陆海空]军|政府|基因|'
        r'global|market(?:ing)?|real estate|system|房地产|系统|兴业)\b',
        re.IGNORECASE)

    # Compile regex pattern for matching common words in company names
    COMPANY_COMMON_WORDS_REGEX = re.compile(
        r"{0}|{1}".format(
            '|'.join(COMPANY_COMMON_WORDS['zh']),
            rf"(\b|^)({'|'.join(COMPANY_COMMON_WORDS['en'])})(\.|\b|$)"
        ), re.I
    )
    # Define ranges for CJK (Chinese, Japanese, Korean) characters
    CJK_CHARACTERS = r'\u1100-\u11ff\u2e80-\u2fff\u3040-\u31ff\u3400-\u9fff\ua960-\ua97f\uac00-\ud7ff\uf900-\ufaff'

    # Regex to match CJK characters or numeric patterns
    CJK_OR_NUMERIC_REGEX = re.compile(
        rf"(?P<cjk>[{CJK_CHARACTERS}]+)|(?P<numeric>((?<=^\D)|(?<=[^\W0-9_]|\s))(?<!\b[a-zA-Z])(\d+([\W_]{{0,5}}\d+){{0,5}})(?=($|[^\W0-9_]|\s)))"
    )

    WHITESPACE_REGEX = re.compile(r'\s+')

    # Common prepositions
    PREPOSITION_WORDS = {
        'aboard', 'about', 'above', 'across', 'after', 'against', 'along', 'amid', 'among', 'anti', 'around', 'as',
        'at',
        'before', 'behind', 'below', 'beneath', 'beside', 'besides', 'between', 'beyond', 'but', 'by',
        'concerning', 'considering', 'despite', 'down', 'during', 'except', 'excepting', 'excluding', 'following',
        'for', 'from', 'in', 'inside', 'into', 'like', 'minus', 'near', 'of', 'off', 'on', 'onto', 'opposite',
        'outside', 'over', 'past', 'per', 'plus', 'regarding', 'round', 'since', 'than', 'through', 'till', 'to',
        'toward', 'towards', 'under', 'underneath', 'unlike', 'until', 'up', 'upon', 'versus', 'via', 'with', 'within',
        'without'
    }

    # Common conjunctions
    CONJUNCTIONS_WORDS = {
        "although", "because", "before", "both", "and", '&', "whether", "or", "either", "neither", "nor", "once",
        "just", "so", "as", "if", "then", "rather", "than", "till", 'when', 'where', 'whenever', 'while', 'wherever',
        "such", "so", "that"
    }

    # Common articles
    ARTICLES = {'a', 'an', 'the'}

    # Merge all irrelevant words (prepositions, conjunctions, articles, etc.)
    INLINE_WORDS = {
                       'also', 'am', 'are', 'did', 'furthermore', 'has', 'hence', 'how', 'however', 'includ.',
                       'instead', 'is', 'likewise',
                       'long', 'moreover', 'should', 'similar', 'though', 'thus', 'unless', 'was', 'were', 'what',
                       'which', 'whichever',
                       'why', 'will',
                       # Portugese prepositions
                       'de', 'di', 'em', 'del', 'des', 'do',
                       # 意大利语介词
                       'delle',
                       # Spanish prepositions
                       'con', 'sobre', 'en', 'contra', 'desde', 'entre', 'hacia', 'por', 'la',
                       # French prepositions
                       'apres', 'avant', 'avec', 'chez', 'contre', 'dans', 'depuis', 'derriere', 'devant', 'durant',
                       'envers', 'environ',
                       'jusque', 'malgre', 'par', 'parmi', 'pendant', 'pour', 'sans', 'selon', 'sous', 'suivant', 'sur',
                       'vers'
                   } | PREPOSITION_WORDS | CONJUNCTIONS_WORDS | ARTICLES

    # Common suffixes in company names (e.g. Ltd, Company, etc.)
    ORG_SUFFIX_WORDS = {
        'zh': {'common': ('公司', '集团', '有限公司', '有限责任公司', '股份有限公司'),
               'uncommon': ('总公司', '股份', '控股', '责任', '有限', '企业', '协会', '合作社', '株式会社')},
        'en': {'common': ('company', 'group', 'corporation', 'incorporated', 'enterprise', 'enterprises',
                          'co', 'inc', 'corp', 'ltd', 'llc', 'se', 'pvt'),
               'uncommon': ('corporation limited', 'companies', 'worldwide', 'limited', "holding", "holdings",
                            'com', 'gmbh', 'ag', 'plc', 'sal', 'spa', r's\.p\.a', 'sab cv', 'sa', 'nv', r'n\.v', 'lp',
                            'sro',
                            'kg', 'aktiengesellschaft', 'de cv', 'ltda', "group of companies")}
    }
    REGEX_COMPANY_DROP_INVALID = re.compile(r'[~$()@—－（）√《》:;：；.、•·‧・，*?+&|,/\[\\]|{}\t\n"]|([-]+\s|\s[-]+)')
    # Compile regular expressions for matching company suffixes
    ORG_SUFFIX_REGEX = re.compile(
        r"{0}|{1}".format(
            '|'.join([rf'{suf}' for suf in ORG_SUFFIX_WORDS['zh']['uncommon'] + ORG_SUFFIX_WORDS['zh']['common']]),
            rf"(?:\b(?:{'|'.join([suf for suf in ORG_SUFFIX_WORDS['en']['uncommon'] + ORG_SUFFIX_WORDS['en']['common']])})(?:\.|$))"
        ),
        re.IGNORECASE
    )

    def __init__(self, company_data_file: str):
        self.company_data_file = company_data_file
        self.matcher = NameMatcher(remove_ascii=False, punctuations=False)
        self.matching_data = None

    def load_company_data(self) -> pd.DataFrame:
        """
        Load the company data from a JSON Lines (.jsonl) file into a pandas DataFrame.
        Ensures aliases and required search strings are properly formatted.

        Returns:
        - pd.DataFrame: DataFrame containing company data including names and aliases.
        """
        companies = []
        with open(self.company_data_file, 'r', encoding='utf-8') as f:
            for line in f:
                company = json.loads(line)
                company['aliases'] = company.get('aliases', [])  # 确保别名存在
                companies.append(company)
        return pd.DataFrame(companies)

    def tokenize(self, text: str) -> str:
        """
        Tokenize the input text using Jieba for Chinese words and return the tokenized string.

        Parameters:
        - text (str): The text to tokenize (usually the company name).

        Returns:
        - str: Tokenized text.
        """
        # Use jieba to segment the text and concatenate the results with spaces into a string and return it
        return " ".join(jieba.cut(text))

    def search_string_regulator(self, name: str, return_str=True, is_company_name=False) -> str:
        """
        Regulate the search string to remove meaningless words and symbols. Handles both Chinese and English.

        :param name: The company name or string to regulate.
        :param return_str: If True, return the result as a string; otherwise, return as a list of words.
        :param is_company_name: Special flag to avoid removing certain common words for company names.
        """
        # Remove meaningless parts of the company name
        extracted_name = re.sub(r'[^\w\s]', '', name)  # Adjust this regex as per your needs
        if not extracted_name:
            return '' if return_str else []

        words = []
        # Split based on spaces or known separators
        for segment in extracted_name.split():
            if not segment:
                continue
            # If the segment is Chinese, treat it as a whole word
            if self.contains_chinese(segment):
                words.append(segment)
            # If it's a meaningful English word, keep it
            else:
                segment = segment.lower()
                if segment not in words or is_company_name:
                    words.append(segment)

        # Return as a single string or a list
        return ' '.join(words) if return_str else words

    def regulate_english_asians_mixed_string(self, text: str) -> str:
        """
        This function handles the regulation of mixed English and Asian (CJK) strings.
        It splits the input string into English, numbers, and CJK segments, then rejoins them.
        """
        # 调用 split_english_number_cjk 方法并传入正确的参数
        return ' '.join(self.split_english_number_cjk(text, separate_return=False))

    def split_english_number_cjk(self, text: str, separate_return: bool = True):
        """
        A utility function that splits text into English, numbers, and CJK (Chinese, Japanese, Korean) characters.
        This is a basic version to simulate the behavior.
        """
        import re
        CJK_REGEX = r'[\u4e00-\u9fff\uf900-\ufaff]+'  # Simplified regex to match CJK characters.
        EN_NUM_REGEX = r'[a-zA-Z0-9]+'

        cjk_parts = re.findall(CJK_REGEX, text)
        en_num_parts = re.findall(EN_NUM_REGEX, text)

        if separate_return:
            return en_num_parts, cjk_parts  # Return both parts separately
        else:
            return en_num_parts + cjk_parts  # Concatenate the parts if not separating

    def prepare_matching_data(self, companies: pd.DataFrame) -> pd.DataFrame:
        """
        Prepare data for matching, ensuring aliases and required search strings are correctly loaded as individual entries.
        """
        rows = []
        for _, company in companies.iterrows():
            # Process required search strings, ensure it's a list
            required_search_strings = company.get("requiredSearchStrings", [])
            if not isinstance(required_search_strings, list):
                required_search_strings = []  # If it's not a list, convert to empty list

            # Ensure aliases are also a list
            aliases = company.get("aliases", [])
            if not isinstance(aliases, list):
                aliases = []  # If it's not a list, convert to empty list

            # Combine required search strings and aliases into one list
            full_alias_list = required_search_strings + aliases

            # For each alias, create a separate row with all aliases as alias_name
            for alias in full_alias_list:
                rows.append({
                    "name": alias,
                    "companyName": company.get("companyName", ""),
                    "alias_name": full_alias_list,  # Store the full alias list
                    "is_alias": alias in aliases  # Mark if this is an alias
                })

        return pd.DataFrame(rows)

    def prepare_data_for_matching(self, companies: pd.DataFrame):
        """
        Prepare the company data for matching by tokenizing the names and loading them into the matcher.
        """
        companies['name'] = companies['name'].apply(self.tokenize)  # Tokenize company names
        self.matcher.load_and_process_master_data(column="name", df_matching_data=companies)

    def match_user_input(self, user_input: str, expected_name: str) -> str:
        """
        Match user input against the company names, then validate against expected name.
        """

        # Input is NaN or non-string, convert it to an empty string
        if not isinstance(user_input, str) or pd.isna(user_input):
            user_input = ''

        # Call interpunction_regulator to process the symbols entered by the user
        user_input = self.interpunction_regulator(user_input)  # 处理符号
        if not user_input:
            return ''

        # Normalize and tokenize the input
        user_input_segmented = self.company_words_regulator(user_input)
        user_input_segmented = " ".join(jieba.cut(user_input_segmented))
        to_be_matched = pd.DataFrame({"name": [user_input_segmented]})

        # Matches the name entered by the user
        result = self.matcher.match_names(to_be_matched, column_matching="name")

        # No match, returns an empty string
        if result.empty:
            return ''

        # Get the best matching results
        best_match = result.iloc[0]["match_name"]
        match_score = result.iloc[0]["score"]

        # If the match score exceeds 70%, it is considered a successful match.
        if match_score >= 70:
            # 通过 best_match 查找公司名称和别名
            matched_row = self.matching_data[self.matching_data['name'] == best_match]
            if matched_row.empty:
                return ''

            # 获取公司名称和别名信息
            company_name = matched_row.iloc[0]['companyName']
            aliases = matched_row.iloc[0]['alias_name']  # 这现在是一个包含所有别名的列表

            # 处理 expected_name，防止 NaN 导致错误
            if pd.isna(expected_name):
                expected_name = ""  # 如果是 NaN，替换为空字符串
            else:
                expected_name = expected_name.lower()

            # 检查 expected_name 是否在别名中
            if expected_name and expected_name in [alias.lower() for alias in aliases]:
                return expected_name

            # 如果 expected_name 和 company_name 匹配，返回公司名称
            if expected_name == company_name.lower():
                return company_name

            # 如果没有匹配到 expected_name，默认返回公司名称
            return company_name
        else:
            return ''

    @staticmethod
    def contains_chinese(text: str) -> bool:
        """
        Check if the input text contains any Chinese characters.
        """
        CHINESE_CHARACTERS = r'\u2e80-\u2fff\u31c0-\u31ef\u3400-\u9fff\uf900-\ufaff'
        return bool(re.search(f"[{CHINESE_CHARACTERS}]", text))

    def company_words_regulator(self, company_name: str) -> str:
        """
        Clean up and regulate the company name by applying normalization rules for circumflexes, symbols,
        removing unnecessary company suffixes, and common words.
        """
        # Step 1: Normalize circumflex letters and process interpunction
        search_str = self.interpunction_regulator(NameNormalizer.circumflex_regulator(company_name))

        if not search_str:
            return ''

        # Step 2: Check if the string contains Chinese and process accordingly
        if self.contains_chinese(search_str):
            if not re.search(r"[a-zA-Z]", search_str):
                search_str = search_str.replace(' ', '')

        # Step 3: Remove company suffixes like 'inc', 'corp', etc., potentially twice to handle two occurrences
        search_str_drop = self.WHITESPACE_REGEX.sub(' ', self.ORG_SUFFIX_REGEX.sub('', self.ORG_SUFFIX_REGEX.sub('',
                                                                                                                 search_str).strip())).strip()

        # The string becomes empty after removing the suffixes, return an empty string
        if not search_str_drop:
            return ''

        # Step 4: Remove stock exchange abbreviations, e.g., 'SSE plc', 'NYSE', 'NASDAQ'
        search_str_drop_bourse = self.COMPANY_COMMON_WORDS_REGEX.sub(' ', search_str_drop).strip()
        if search_str_drop_bourse:
            search_str_drop = search_str_drop_bourse

        # Step 5: Remove common company words like 'technology', 'research', etc.
        search_str_drop_common_word = self.WHITESPACE_REGEX.sub(' ', self.COMPANY_COMMON_WORDS_REGEX.sub('',
                                                                                                         search_str_drop)).strip()
        if search_str_drop_common_word:
            search_str_drop = search_str_drop_common_word

        # Step 6: Ensure that removing common words doesn't leave just a keyword, e.g., 'Corporation Bank'
        if (r_ := self.COMPANY_KEYWORDS_REGEX.search(search_str_drop)) and r_.end() - r_.start() > len(
                search_str_drop) - 2:
            pass
        elif search_str == search_str_drop:
            pass
        else:
            search_str = search_str_drop

        # Step 7: Further regulate the search string and convert to lowercase
        search_str_regulate = self.search_string_regulator(search_str, is_company_name=True).lower()
        if search_str_regulate != search_str:
            search_str = search_str_regulate

        # Step 8: Remove invalid characters and filter valid words
        valid_words = []
        for word_ in search_str.split(' '):
            word_ = self.REGEX_COMPANY_DROP_INVALID.sub(' ', word_).strip().lower()
            if word_:
                valid_words.append(word_)

        # Return the cleaned and processed company name
        return ' '.join(valid_words)

    # Direct deletion symbols: '-' and '/'
    ERASE_SYMBOLS = re.compile(r'[-/]')



    def interpunction_regulator(self, name: str) -> str:
        """
        Regulates punctuation and symbols in the company name by:
        - Removing or replacing certain symbols with spaces.
        - Replacing special characters like '&' and '@' with words ('togetherwith', 'locatedat').
        - Handling special cases like sequential single letters and 'the' followed by a short word.
        """
        is_chinese = self.contains_chinese(name)

        if not is_chinese:
            # If name contains specific symbols, split and take the first word for further matching
            if name_words := re.split(r'[|]+', name):
                name = name_words[0]

        # Replace '&' with 'togetherwith' and '@' with 'locatedat'
        if and_match := re.search(r'&', name):
            name = name.replace('&', 'togetherwith')
        if at_match := re.search(r'@', name):
            name = name.replace('@', 'locatedat')

        # Replace specific symbols with spaces
        name = REPLACE_TO_SPACE_SYMBOLS.sub(' ', name)

        # Use ERASE_OR_REPLACE_TO_SPACE_SYMBOLS instead of ERASE_SYMBOLS
        if is_chinese:
            name = ERASE_OR_REPLACE_TO_SPACE_SYMBOLS.sub(' ', name).strip()
        else:
            name = ERASE_OR_REPLACE_TO_SPACE_SYMBOLS.sub('', name).strip()

        # Handle special cases for names starting with 'the' followed by a short word
        if the_match := THE_SPECIAL_ENTITY_NAME_REGEX.search(name):
            connect_str = the_match.group(1)
            name = name.replace(connect_str, re.sub(r'[\s]+', 'thespecial', connect_str))

        # Merge sequential single letters into one
        before_merge_name = name
        name = ''
        while True:
            if not (single_letter_sequential := SINGLE_LETTER_SEQUENTIAL_REGEX.search(before_merge_name)):
                name += before_merge_name.strip()
                break
            name += before_merge_name[
                    :single_letter_sequential.start()].strip() + ' ' + single_letter_sequential.group().replace(' ',
                                                                                                                '') + ' '
            before_merge_name = before_merge_name[single_letter_sequential.end():]

        # Drop single lowercase letters, but not if they are at the start or end
        name = SINGLE_LOWERCASE_REGEX.sub(' ', name)

        if is_chinese:
            name = self.regulate_english_asians_mixed_string(name)

        return name if name else None


def main():
    """
    Main function to initialize CompanyMatcher, load company data, and perform company name matching.
    """
    # Initialize CompanyMatcher and pass in the path to the company data file
    company_matcher = CompanyMatcher('bd_companies_international.jsonl')

    # Load Company Data
    companies = company_matcher.load_company_data()

    # Prepare matching data and organize company name and alias information into a format that can be matched
    matching_data = company_matcher.prepare_matching_data(companies)
    company_matcher.matching_data = matching_data

    # Prepare company data for matching (e.g. word segmentation)
    company_matcher.prepare_data_for_matching(matching_data)

    # Get input from the user and match the company name
    user_input = input("Enter a company name for matching: ")

    # Using a matcher to perform company name matching
    matched_company = company_matcher.match_user_input(user_input, expected_name=None)

    if matched_company:
        print(f"Matched company: {matched_company}")
    else:
        print("")


if __name__ == "__main__":
    main()


In [ ]:
#V11B
#Can Use V10B Unittest
#Can Use V10B calculation and graph code
#Add suffix process
import re
import pandas as pd
import json
import jieba
from name_matching.name_matcher import NameMatcher

# A regular expression to match and replace certain symbols with spaces, including slashes, percentage signs, exclamations, punctuation, hyphens, and special symbols.
ERASE_OR_REPLACE_TO_SPACE_SYMBOLS = re.compile(r'''[/\\%!！.?？^*·•⦁\-]''')

# A regular expression to match entity names that start with "the" followed by alphanumeric characters, case-insensitive.
THE_SPECIAL_ENTITY_NAME_REGEX = re.compile(r'^(the\s+[a-z0-9]+)$', re.IGNORECASE)

# A regular expression to match sequences of single letters separated by spaces (e.g., "a b c") and find those patterns, case-insensitive.
SINGLE_LETTER_SEQUENTIAL_REGEX = re.compile(rf'(\b[a-z]\b)(\s+[a-z]\b)+', re.IGNORECASE)

# A regular expression to match single lowercase letters surrounded by spaces (e.g., " a ").
SINGLE_LOWERCASE_REGEX = re.compile(rf'\s+[a-z]\s+')

# A regular expression to match and remove invalid symbols in company names, such as punctuation, special characters, and spaces around hyphens.
REGEX_COMPANY_DROP_INVALID = re.compile(r'[~$()@—－（）√《》:;：；.、•·‧・，*?+&|,/\[\\]|{}\t\n"]|([-]+\s|\s[-]+)')

# A regular expression to match symbols that should be replaced with spaces, including ampersands, question marks, asterisks, parentheses, commas, and hyphens.
REPLACE_TO_SPACE_SYMBOLS = re.compile(r'[&?#*|=_+()（）、,，–]')

class NameNormalizer:
    """
    This class handles the normalization of names, including circumflex letters and symbols.
    """

    # Pairing circumflex letters with their normalized counterparts
    CIRCUMFLEX_PAIRS = (
        (r'àâáãāåäǎ', 'a'), (r'ÀÂÁÃĀÅÄǍ', 'A'), (r'ß', 'b'), (r'čćç', 'c'), (r'ČÇ', 'C'),
        (r'ðđ', 'd'), (r'ÐĐ', 'D'), (r'éěèêëėệē', 'e'), (r'ÉÈÊËĖỆĒ', 'E'), (r'ğ', 'g'),
        (r'Ğ', 'G'), (r'îïíīịìǐĩı', 'i'), (r'İÎÏÍĪỊÌǏ', 'I'), (r'Ł', 'L'), (r'ł', 'l'),
        (r'ñňń', 'n'), (r'ÑŇŃ', 'N'), (r'ôöōồǒóòøőõ', 'o'), (r'ÔÖŌỒÓÒØŐÕ', 'O'), (r'ŕ', 'r'),
        (r'şš', 's'), (r'ŠŞ', 'S'), (r'ť', 't'), (r'ûùüúǔưū', 'u'), (r'ÛÙÜÚƯŪ', 'U'),
        (r'ŵ', 'w'), (r'ý', 'y'), (r'Ý', 'Y'), (r'žź', 'z'), (r'ŽŹ', 'Z'), (r'æ', 'ae'), (r'Æ', 'AE')
    )

    # Compile regex for circumflex replacements
    CIRCUMFLEX_REGEX_PAIRS = tuple(
        (re.compile(rf'[{regex_}]'), format_) for (regex_, format_) in CIRCUMFLEX_PAIRS
    )

    @classmethod
    def circumflex_regulator(cls, name: str) -> str:
        """
        Normalize circumflex letters in the name, e.g., 'é' becomes 'e'.
        If name is not a string, return an empty string.
        """
        if not isinstance(name, str) or pd.isna(name):
            return ''

        # Replace circumflex letters using pre-compiled regex pairs
        for regex_, format_character in cls.CIRCUMFLEX_REGEX_PAIRS:
            name = regex_.sub(format_character, name)
        return name


class CompanyMatcher:
    """
    This class handles company name matching using various techniques like tokenization, symbol regulation,
    and name normalization.
    """
    # Define common words for both English and Chinese
    COMPANY_COMMON_WORDS = {
        "en": {
            "laboratory", "laboratories", "lab", "labs", "research", "technology", "technologies",
            "technical", "tech", "sci", "science"
        },
        "zh": {
            "实验室", "研究院", "技术", "科技"
        }
    }
    COMPANY_KEYWORDS_REGEX = re.compile(
        r'\b(bank|college|university|education|energy|finance|army|air foce|navy|industry|'
        r'hospital|hotel|institutes?|institution|petroleum|oil|health|electronic|commercial|'
        r'environmental|gover(?:nor?|ment)|state|estado|procter|reliance|genome|software|hardware|agency|'
        r'(?:sou|nor)th(?:[-\s]?(?:ea|we)st(?:ern)?)?|[东西南北]方|[东西][南北][方]?|环境|'
        r'银行|软件|硬件|大学|机构|研究院|学院|教育|协会|能源|金融|旅馆|医院|石油|电子|工业|[陆海空]军|政府|基因|'
        r'global|market(?:ing)?|real estate|system|房地产|系统|兴业)\b',
        re.IGNORECASE)

    # Compile regex pattern for matching common words in company names
    COMPANY_COMMON_WORDS_REGEX = re.compile(
        r"{0}|{1}".format(
            '|'.join(COMPANY_COMMON_WORDS['zh']),
            rf"(\b|^)({'|'.join(COMPANY_COMMON_WORDS['en'])})(\.|\b|$)"
        ), re.I
    )
    # Define ranges for CJK (Chinese, Japanese, Korean) characters
    CJK_CHARACTERS = r'\u1100-\u11ff\u2e80-\u2fff\u3040-\u31ff\u3400-\u9fff\ua960-\ua97f\uac00-\ud7ff\uf900-\ufaff'

    # Regex to match CJK characters or numeric patterns
    CJK_OR_NUMERIC_REGEX = re.compile(
        rf"(?P<cjk>[{CJK_CHARACTERS}]+)|(?P<numeric>((?<=^\D)|(?<=[^\W0-9_]|\s))(?<!\b[a-zA-Z])(\d+([\W_]{{0,5}}\d+){{0,5}})(?=($|[^\W0-9_]|\s)))"
    )

    WHITESPACE_REGEX = re.compile(r'\s+')

    # Common prepositions
    PREPOSITION_WORDS = {
        'aboard', 'about', 'above', 'across', 'after', 'against', 'along', 'amid', 'among', 'anti', 'around', 'as',
        'at',
        'before', 'behind', 'below', 'beneath', 'beside', 'besides', 'between', 'beyond', 'but', 'by',
        'concerning', 'considering', 'despite', 'down', 'during', 'except', 'excepting', 'excluding', 'following',
        'for', 'from', 'in', 'inside', 'into', 'like', 'minus', 'near', 'of', 'off', 'on', 'onto', 'opposite',
        'outside', 'over', 'past', 'per', 'plus', 'regarding', 'round', 'since', 'than', 'through', 'till', 'to',
        'toward', 'towards', 'under', 'underneath', 'unlike', 'until', 'up', 'upon', 'versus', 'via', 'with', 'within',
        'without'
    }

    # Common conjunctions
    CONJUNCTIONS_WORDS = {
        "although", "because", "before", "both", "and", '&', "whether", "or", "either", "neither", "nor", "once",
        "just", "so", "as", "if", "then", "rather", "than", "till", 'when', 'where', 'whenever', 'while', 'wherever',
        "such", "so", "that"
    }

    # Common articles
    ARTICLES = {'a', 'an', 'the'}

    # Merge all irrelevant words (prepositions, conjunctions, articles, etc.)
    INLINE_WORDS = {
                       'also', 'am', 'are', 'did', 'furthermore', 'has', 'hence', 'how', 'however', 'includ.',
                       'instead', 'is', 'likewise',
                       'long', 'moreover', 'should', 'similar', 'though', 'thus', 'unless', 'was', 'were', 'what',
                       'which', 'whichever',
                       'why', 'will',
                       # Portugese prepositions
                       'de', 'di', 'em', 'del', 'des', 'do',
                       # 意大利语介词
                       'delle',
                       # Spanish prepositions
                       'con', 'sobre', 'en', 'contra', 'desde', 'entre', 'hacia', 'por', 'la',
                       # French prepositions
                       'apres', 'avant', 'avec', 'chez', 'contre', 'dans', 'depuis', 'derriere', 'devant', 'durant',
                       'envers', 'environ',
                       'jusque', 'malgre', 'par', 'parmi', 'pendant', 'pour', 'sans', 'selon', 'sous', 'suivant', 'sur',
                       'vers'
                   } | PREPOSITION_WORDS | CONJUNCTIONS_WORDS | ARTICLES

    # Common suffixes in company names (e.g. Ltd, Company, etc.)
    ORG_SUFFIX_WORDS = {
        'zh': {'common': ('公司', '集团', '有限公司', '有限责任公司', '股份有限公司'),
               'uncommon': ('总公司', '股份', '控股', '责任', '有限', '企业', '协会', '合作社', '株式会社')},
        'en': {'common': ('company', 'group', 'corporation', 'incorporated', 'enterprise', 'enterprises',
                          'co', 'inc', 'corp', 'ltd', 'llc', 'se', 'pvt'),
               'uncommon': ('corporation limited', 'companies', 'worldwide', 'limited', "holding", "holdings",
                            'com', 'gmbh', 'ag', 'plc', 'sal', 'spa', r's\.p\.a', 'sab cv', 'sa', 'nv', r'n\.v', 'lp',
                            'sro',
                            'kg', 'aktiengesellschaft', 'de cv', 'ltda', "group of companies")}
    }
    REGEX_COMPANY_DROP_INVALID = re.compile(r'[~$()@—－（）√《》:;：；.、•·‧・，*?+&|,/\[\\]|{}\t\n"]|([-]+\s|\s[-]+)')
    # Compile regular expressions for matching company suffixes
    ORG_SUFFIX_REGEX = re.compile(
        r"{0}|{1}".format(
            '|'.join([rf'{suf}' for suf in ORG_SUFFIX_WORDS['zh']['uncommon'] + ORG_SUFFIX_WORDS['zh']['common']]),
            rf"(?:\b(?:{'|'.join([suf for suf in ORG_SUFFIX_WORDS['en']['uncommon'] + ORG_SUFFIX_WORDS['en']['common']])})(?:\.|$))"
        ),
        re.IGNORECASE
    )

    def __init__(self, company_data_file: str):
        self.company_data_file = company_data_file
        self.matcher = NameMatcher(remove_ascii=False, punctuations=False)
        self.matching_data = None

    def load_company_data(self) -> pd.DataFrame:
        """
        Load the company data from a JSON Lines (.jsonl) file into a pandas DataFrame.
        Ensures aliases and required search strings are properly formatted.

        Returns:
        - pd.DataFrame: DataFrame containing company data including names and aliases.
        """
        companies = []
        with open(self.company_data_file, 'r', encoding='utf-8') as f:
            for line in f:
                company = json.loads(line)
                company['aliases'] = company.get('aliases', [])  # 确保别名存在
                companies.append(company)
        return pd.DataFrame(companies)

    def tokenize(self, text: str) -> str:
        """
        Tokenize the input text using Jieba for Chinese words and return the tokenized string.

        Parameters:
        - text (str): The text to tokenize (usually the company name).

        Returns:
        - str: Tokenized text.
        """
        # Use jieba to segment the text and concatenate the results with spaces into a string and return it
        return " ".join(jieba.cut(text))

    def search_string_regulator(self, name: str, return_str=True, is_company_name=False) -> str:
        """
        Regulate the search string to remove meaningless words and symbols. Handles both Chinese and English.

        :param name: The company name or string to regulate.
        :param return_str: If True, return the result as a string; otherwise, return as a list of words.
        :param is_company_name: Special flag to avoid removing certain common words for company names.
        """
        # Remove meaningless parts of the company name
        extracted_name = re.sub(r'[^\w\s]', '', name)  # Adjust this regex as per your needs
        if not extracted_name:
            return '' if return_str else []

        words = []
        # Split based on spaces or known separators
        for segment in extracted_name.split():
            if not segment:
                continue
            # If the segment is Chinese, treat it as a whole word
            if self.contains_chinese(segment):
                words.append(segment)
            # If it's a meaningful English word, keep it
            else:
                segment = segment.lower()
                if segment not in words or is_company_name:
                    words.append(segment)

        # Return as a single string or a list
        return ' '.join(words) if return_str else words

    def regulate_english_asians_mixed_string(self, text: str) -> str:
        """
        This function handles the regulation of mixed English and Asian (CJK) strings.
        It splits the input string into English, numbers, and CJK segments, then rejoins them.
        """
        # 调用 split_english_number_cjk 方法并传入正确的参数
        return ' '.join(self.split_english_number_cjk(text, separate_return=False))

    def split_english_number_cjk(self, text: str, separate_return: bool = True):
        """
        A utility function that splits text into English, numbers, and CJK (Chinese, Japanese, Korean) characters.
        This is a basic version to simulate the behavior.
        """
        import re
        CJK_REGEX = r'[\u4e00-\u9fff\uf900-\ufaff]+'  # Simplified regex to match CJK characters.
        EN_NUM_REGEX = r'[a-zA-Z0-9]+'

        cjk_parts = re.findall(CJK_REGEX, text)
        en_num_parts = re.findall(EN_NUM_REGEX, text)

        if separate_return:
            return en_num_parts, cjk_parts  # Return both parts separately
        else:
            return en_num_parts + cjk_parts  # Concatenate the parts if not separating

    def prepare_matching_data(self, companies: pd.DataFrame) -> pd.DataFrame:
        """
        Prepare data for matching, ensuring aliases and required search strings are correctly loaded as individual entries.
        """
        rows = []
        for _, company in companies.iterrows():
            # Process required search strings, ensure it's a list
            required_search_strings = company.get("requiredSearchStrings", [])
            if not isinstance(required_search_strings, list):
                required_search_strings = []  # If it's not a list, convert to empty list

            # Ensure aliases are also a list
            aliases = company.get("aliases", [])
            if not isinstance(aliases, list):
                aliases = []  # If it's not a list, convert to empty list

            # Combine required search strings and aliases into one list
            full_alias_list = required_search_strings + aliases

            # For each alias, create a separate row with all aliases as alias_name
            for alias in full_alias_list:
                rows.append({
                    "name": alias,
                    "companyName": company.get("companyName", ""),
                    "alias_name": full_alias_list,  # Store the full alias list
                    "is_alias": alias in aliases  # Mark if this is an alias
                })

        return pd.DataFrame(rows)

    def prepare_data_for_matching(self, companies: pd.DataFrame):
        """
        Prepare the company data for matching by tokenizing the names and loading them into the matcher.
        """
        companies['name'] = companies['name'].apply(self.tokenize)  # Tokenize company names
        self.matcher.load_and_process_master_data(column="name", df_matching_data=companies)

    def match_user_input(self, user_input: str, expected_name: str, threshold: int = 70) -> str:
        """
        Match user input against the company names, then validate against expected name.
        """

        # Input is NaN or non-string, convert it to an empty string
        if not isinstance(user_input, str) or pd.isna(user_input):
            user_input = ''

        # Call interpunction_regulator to process the symbols entered by the user
        user_input = self.interpunction_regulator(user_input)  # 处理符号
        if not user_input:
            return ''

        # Normalize and tokenize the input
        user_input_segmented = self.company_words_regulator(user_input)
        user_input_segmented = " ".join(jieba.cut(user_input_segmented))
        to_be_matched = pd.DataFrame({"name": [user_input_segmented]})

        # Matches the name entered by the user
        result = self.matcher.match_names(to_be_matched, column_matching="name")

        # No match, returns an empty string
        if result.empty:
            return ''

        # Get the best matching results
        best_match = result.iloc[0]["match_name"]
        match_score = result.iloc[0]["score"]

        # If the match score exceeds the threshold, it is considered a successful match.
        if match_score >= threshold:
            # 通过 best_match 查找公司名称和别名
            matched_row = self.matching_data[self.matching_data['name'] == best_match]
            if matched_row.empty:
                return ''

            # 获取公司名称和别名信息
            company_name = matched_row.iloc[0]['companyName']
            aliases = matched_row.iloc[0]['alias_name']  # 这现在是一个包含所有别名的列表

            # 处理 expected_name，防止 NaN 导致错误
            if pd.isna(expected_name):
                expected_name = ""  # 如果是 NaN，替换为空字符串
            else:
                expected_name = expected_name.lower()

            # 检查 expected_name 是否在别名中
            if expected_name and expected_name in [alias.lower() for alias in aliases]:
                return expected_name

            # 如果 expected_name 和 company_name 匹配，返回公司名称
            if expected_name == company_name.lower():
                return company_name

            # 如果没有匹配到 expected_name，默认返回公司名称
            return company_name
        else:
            return ''

    @staticmethod
    def contains_chinese(text: str) -> bool:
        """
        Check if the input text contains any Chinese characters.
        """
        CHINESE_CHARACTERS = r'\u2e80-\u2fff\u31c0-\u31ef\u3400-\u9fff\uf900-\ufaff'
        return bool(re.search(f"[{CHINESE_CHARACTERS}]", text))

    def company_words_regulator(self, company_name: str) -> str:
        """
        Clean up and regulate the company name by applying normalization rules for circumflexes, symbols,
        removing unnecessary company suffixes, and common words.
        """
        # Step 1: Normalize circumflex letters and process interpunction
        search_str = self.interpunction_regulator(NameNormalizer.circumflex_regulator(company_name))

        if not search_str:
            return ''

        # Step 2: Check if the string contains Chinese and process accordingly
        if self.contains_chinese(search_str):
            if not re.search(r"[a-zA-Z]", search_str):
                search_str = search_str.replace(' ', '')

        # Step 3: Remove company suffixes like 'inc', 'corp', etc., potentially twice to handle two occurrences
        search_str_drop = self.WHITESPACE_REGEX.sub(' ', self.ORG_SUFFIX_REGEX.sub('', self.ORG_SUFFIX_REGEX.sub('',
                                                                                                                 search_str).strip())).strip()

        # The string becomes empty after removing the suffixes, return an empty string
        if not search_str_drop:
            return ''

        # Step 4: Remove stock exchange abbreviations, e.g., 'SSE plc', 'NYSE', 'NASDAQ'
        search_str_drop_bourse = self.COMPANY_COMMON_WORDS_REGEX.sub(' ', search_str_drop).strip()
        if search_str_drop_bourse:
            search_str_drop = search_str_drop_bourse

        # Step 5: Remove common company words like 'technology', 'research', etc.
        search_str_drop_common_word = self.WHITESPACE_REGEX.sub(' ', self.COMPANY_COMMON_WORDS_REGEX.sub('',
                                                                                                         search_str_drop)).strip()
        if search_str_drop_common_word:
            search_str_drop = search_str_drop_common_word

        # Step 6: Ensure that removing common words doesn't leave just a keyword, e.g., 'Corporation Bank'
        if (r_ := self.COMPANY_KEYWORDS_REGEX.search(search_str_drop)) and r_.end() - r_.start() > len(
                search_str_drop) - 2:
            pass
        elif search_str == search_str_drop:
            pass
        else:
            search_str = search_str_drop

        # Step 7: Further regulate the search string and convert to lowercase
        search_str_regulate = self.search_string_regulator(search_str, is_company_name=True).lower()
        if search_str_regulate != search_str:
            search_str = search_str_regulate

        # Step 8: Remove invalid characters and filter valid words
        valid_words = []
        for word_ in search_str.split(' '):
            word_ = self.REGEX_COMPANY_DROP_INVALID.sub(' ', word_).strip().lower()
            if word_:
                valid_words.append(word_)

        # Return the cleaned and processed company name
        return ' '.join(valid_words)

    # Direct deletion symbols: '-' and '/'
    ERASE_SYMBOLS = re.compile(r'[-/]')



    def interpunction_regulator(self, name: str) -> str:
        """
        Regulates punctuation and symbols in the company name by:
        - Removing or replacing certain symbols with spaces.
        - Replacing special characters like '&' and '@' with words ('togetherwith', 'locatedat').
        - Handling special cases like sequential single letters and 'the' followed by a short word.
        """
        is_chinese = self.contains_chinese(name)

        if not is_chinese:
            # If name contains specific symbols, split and take the first word for further matching
            if name_words := re.split(r'[|]+', name):
                name = name_words[0]

        # Replace '&' with 'togetherwith' and '@' with 'locatedat'
        if and_match := re.search(r'&', name):
            name = name.replace('&', 'togetherwith')
        if at_match := re.search(r'@', name):
            name = name.replace('@', 'locatedat')

        # Replace specific symbols with spaces
        name = REPLACE_TO_SPACE_SYMBOLS.sub(' ', name)

        # Use ERASE_OR_REPLACE_TO_SPACE_SYMBOLS instead of ERASE_SYMBOLS
        if is_chinese:
            name = ERASE_OR_REPLACE_TO_SPACE_SYMBOLS.sub(' ', name).strip()
        else:
            name = ERASE_OR_REPLACE_TO_SPACE_SYMBOLS.sub('', name).strip()

        # Handle special cases for names starting with 'the' followed by a short word
        if the_match := THE_SPECIAL_ENTITY_NAME_REGEX.search(name):
            connect_str = the_match.group(1)
            name = name.replace(connect_str, re.sub(r'[\s]+', 'thespecial', connect_str))

        # Merge sequential single letters into one
        before_merge_name = name
        name = ''
        while True:
            if not (single_letter_sequential := SINGLE_LETTER_SEQUENTIAL_REGEX.search(before_merge_name)):
                name += before_merge_name.strip()
                break
            name += before_merge_name[
                    :single_letter_sequential.start()].strip() + ' ' + single_letter_sequential.group().replace(' ',
                                                                                                                '') + ' '
            before_merge_name = before_merge_name[single_letter_sequential.end():]

        # Drop single lowercase letters, but not if they are at the start or end
        name = SINGLE_LOWERCASE_REGEX.sub(' ', name)

        if is_chinese:
            name = self.regulate_english_asians_mixed_string(name)

        return name if name else None


def main():
    """
    Main function to initialize CompanyMatcher, load company data, and perform company name matching.
    """
    # Initialize CompanyMatcher and pass in the path to the company data file
    company_matcher = CompanyMatcher('bd_companies_international.jsonl')

    # Load Company Data
    companies = company_matcher.load_company_data()

    # Prepare matching data and organize company name and alias information into a format that can be matched
    matching_data = company_matcher.prepare_matching_data(companies)
    company_matcher.matching_data = matching_data

    # Prepare company data for matching (e.g. word segmentation)
    company_matcher.prepare_data_for_matching(matching_data)

    # Get input from the user and match the company name
    user_input = input("Enter a company name for matching: ")

    # Using a matcher to perform company name matching
    matched_company = company_matcher.match_user_input(user_input, expected_name=None)

    if matched_company:
        print(f"Matched company: {matched_company}")
    else:
        print("")


if __name__ == "__main__":
    main()


In [ ]:
#V10B/V11B graph for precision vs. recall
"""x is recall
   y is precision"""

import pandas as pd
import matplotlib.pyplot as plt

# Plotting Precision vs Recall
data = pd.read_csv('calculate_score.csv')

# 绘制 Precision vs Recall 的图
plt.figure(figsize=(8, 6))
plt.plot(data['Recall'], data['Precision'], marker='o')

# 添加坐标轴标签和标题
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision vs Recall')

# 显示网格
plt.grid(True)

# 显示图表
plt.show()
